In [2]:
# Mến chào cô!

# Em tên: Trần Huỳnh Chí Nguyên.
# MSSV: 23110136.

# Em xin phép nộp bài tập cá nhân môn Trí tuệ Nhân tạo.

# 💡
    # Em đặt Scale màn hình là 100% trên thiết bị cá nhân, nên nếu bị khuất trên máy cô, cô có thể chỉnh scale giúp em.
    # Các thuật toán em đều làm đủ và đặt trong Combobox (nếu bị khuất cô có thể trượt xuống để xem thêm).

# Em xin đính kèm link gitub: 

# Mến chúc cô sức khỏe!

import tkinter as tk
from tkinter import ttk 
import copy
from collections import deque  
from queue import PriorityQueue
import heapq
import random, math
import re
import collections
 
cars_positions = []
canvas_right_global = None
algorithm_frame = None
state_frame = None
algorithm_combobox = None
selected = ""
timer_elapse = 0
timer_running = False
timer_label = None
CELL_SIZE = 45

original_state = [
    [0,0,0,0,0,0,0,0],
    [0,0,0,0,0,0,0,0],
    [0,0,0,0,0,0,0,0],
    [0,0,0,0,0,0,0,0],
    [0,0,0,0,0,0,0,0],
    [0,0,0,0,0,0,0,0],
    [0,0,0,0,0,0,0,0],
    [0,0,0,0,0,0,0,0],
]

goal_state = [
    [1,0,0,0,0,0,0,0],
    [0,0,1,0,0,0,0,0],
    [0,0,0,1,0,0,0,0],
    [0,1,0,0,0,0,0,0],
    [0,0,0,0,0,0,1,0],
    [0,0,0,0,1,0,0,0],
    [0,0,0,0,0,1,0,0],
    [0,0,0,0,0,0,0,1],
]
BOARD_SIZE = len(original_state)

def update_timer():
    global timer_elapse, timer_running, timer_label
    if timer_running:
        timer_elapse+=1
        timer_label.config(text=f"Thời gian: {timer_elapse}s")
        timer_label.after(1000,update_timer)

def reset_game():
    global timer_elapse, timer_running, cars_positions, selected
    timer_elapse = 0
    timer_running = False
    cars_positions = copy.deepcopy(original_state)  
    timer_label.config(text="Thời gian: 0 s")
    draw_board(canvas_right_global, with_cars=False)
    selected = ""
    show_total_state("⚠ Vui lòng chọn 1 thuật toán trong các\n"\
                        "nhóm thuật toán: Local Search để xem\n"\
                        "được lịch sử các trạng thái đã đi qua ⚠")

def BFS():
    N = BOARD_SIZE
    
    # Chuyển goal_state thành tuple of tuples để so sánh
    GOAL_TUPLE = tuple(tuple(row) for row in goal_state)
    node = tuple(tuple(row) for row in original_state)
    
    path = {node: None} 
    frontier = deque([node])
    explored = {node} 

    def count_rooks(board_tuple):
        return sum(row.count(1) for row in board_tuple)

    while frontier:
        node = frontier.popleft()
        
        # 1. KIỂM TRA GOAL STATE: So sánh trực tiếp với ma trận đích
        if node == GOAL_TUPLE:
            # Tái tạo lịch sử đường đi (Logic này giữ nguyên)
            history = []
            current = node
            temp_path = []
            
            while current is not None:
                temp_path.append(current)
                current = path.get(current)
            
            temp_path.reverse()
            current_rooks = -1
            
            for step_board in temp_path:
                num_rooks = count_rooks(step_board)
                if num_rooks > current_rooks:
                    history.append(f"--- Đặt quân Xe thứ {num_rooks} ---")
                    for r_row in step_board:
                        history.append(f"{list(r_row)}")
                    current_rooks = num_rooks
            
            history.append("TÌM THẤY LỜI GIẢI HỢP LỆ!")
            return [list(row) for row in node], history

        # 2. Sinh trạng thái con (chỉ sinh khi chưa đủ N quân)
        k = count_rooks(node)
        if k == N:
            continue 

        next_row = k
        a = [list(row) for row in node]
        
        for j in range(N):
            is_col_free = True
            for r in range(next_row): 
                if a[r][j] == 1:
                    is_col_free = False
                    break
            
            if is_col_free:
                b = [list(row) for row in a]
                b[next_row][j] = 1
                child = tuple(tuple(row) for row in b)
                
                if child not in explored:
                    path[child] = node
                    explored.add(child)
                    frontier.append(child)

    return None, ["Không tìm thấy lời giải."]
    
def DFS():
    N = BOARD_SIZE
    
    # Chuyển goal_state thành tuple of tuples để so sánh
    GOAL_TUPLE = tuple(tuple(row) for row in goal_state)
    node = tuple(tuple(row) for row in original_state)
    
    path = {node: None}
    stack = [node]
    visited = {node} 

    def count_rooks(board_tuple):
        return sum(row.count(1) for row in board_tuple)

    while stack:
        node = stack.pop()
        
        # 1. KIỂM TRA GOAL STATE: So sánh trực tiếp với ma trận đích
        if node == GOAL_TUPLE:
            # Tái tạo lịch sử đường đi (Logic này giữ nguyên)
            history = []
            current = node
            temp_path = []
            
            while current is not None:
                temp_path.append(current)
                current = path.get(current) 
            
            temp_path.reverse()
            current_rooks = -1
            
            for step_board in temp_path:
                num_rooks = count_rooks(step_board)
                if num_rooks > current_rooks:
                    history.append(f"--- Đặt quân Xe thứ {num_rooks} ---")
                    for r_row in step_board:
                        history.append(f"{list(r_row)}")
                    current_rooks = num_rooks
            
            history.append("TÌM THẤY LỜI GIẢI HỢP LỆ!")
            return [list(row) for row in node], history
        
        # 2. Sinh trạng thái con (chỉ sinh khi chưa đủ N quân)
        k = count_rooks(node)
        if k == N:
            continue

        next_row = k
        a = [list(row) for row in node]
        
        # Duyệt ngược (reversed) là tiêu chuẩn trong DFS để tối ưu tốc độ tìm kiếm
        for j in reversed(range(N)): 
            is_col_free = True
            for r in range(next_row): 
                if a[r][j] == 1:
                    is_col_free = False
                    break
            
            if is_col_free:
                b = [list(row) for row in a]
                b[next_row][j] = 1
                child = tuple(tuple(row) for row in b)
                
                if child not in visited:
                    path[child] = node
                    visited.add(child)
                    stack.append(child)

    return None, ["Không tìm thấy lời giải."]
    
def UCS():
    N = BOARD_SIZE
    
    # Chuyển goal_state thành tuple of tuples để so sánh
    GOAL_TUPLE = tuple(tuple(row) for row in goal_state) 
    
    def distance(p1, p2):
        return abs(p1[0] - p2[0]) + abs(p1[1] - p2[1])
        
    def count_rooks(board_tuple):
        return sum(row.count(1) for row in board_tuple)

    original_state_tuple = tuple(tuple(row) for row in original_state)
    
    que = []
    # (cost, state, positions, last_pos)
    heapq.heappush(que, (0, original_state_tuple, [], None)) 
    
    visited = {original_state_tuple: 0}
    path = {original_state_tuple: (None, 0)} 

    while que:
        cost, node, positions, last_pos = heapq.heappop(que)
        
        # 1. Kiểm tra Goal State: SO SÁNH TRỰC TIẾP VỚI GOAL_TUPLE
        if node == GOAL_TUPLE:
            # --- Tái tạo lịch sử đường đi và chi phí ---
            final_board = [list(row) for row in node]
            history = []
            current = node
            temp_path = []
            
            while current is not None:
                parent, step_cost = path.get(current, (None, 0))
                temp_path.append((current, step_cost))
                current = parent
            
            temp_path.reverse()
            
            current_rooks = -1
            total_cost = 0
            for step_board, step_cost in temp_path:
                num_rooks = count_rooks(step_board)
                
                # Chỉ thêm trạng thái khi có quân mới được đặt
                if num_rooks > current_rooks: 
                    total_cost += step_cost
                    history.append(f"--- Đặt quân Xe thứ {num_rooks} (Chi phí: {step_cost}) ---")
                    
                    for r_row in step_board:
                        history.append(f"{list(r_row)}")
                    
                    current_rooks = num_rooks
            
            history.append(f"TÌM THẤY LỜI GIẢI HỢP LỆ! TỔNG CHI PHÍ: {total_cost}")
            return final_board, total_cost, history
        
        # Bỏ qua nếu tìm thấy đường đi rẻ hơn đến trạng thái này
        if cost > visited.get(node, float('inf')):
             continue
        
        # ... (Phần còn lại của logic sinh trạng thái con) ...
        k = count_rooks(node)
        next_row = k 
        
        # DỪNG SINH TRẠNG THÁI NẾU ĐÃ ĐẶT ĐỦ N QUÂN
        # Nếu goal_state là trạng thái N quân cuối cùng, ta cần đảm bảo
        # rằng ta không cố gắng đặt thêm quân sau khi đã đặt N-1 quân.
        if k == N:
            continue

        a = [list(row) for row in node]
        
        for j in range(N):
            is_col_free = True
            for r in range(next_row): 
                if a[r][j] == 1:
                    is_col_free = False
                    break
            
            if is_col_free:
                b = [list(row) for row in a]
                b[next_row][j] = 1

                child = tuple(tuple(row) for row in b)
                new_position = (next_row, j)
                
                # Tính chi phí
                if not positions:
                    step_cost = distance((0, 0), new_position)
                else:
                    step_cost = distance(positions[-1], new_position)
                
                new_cost = cost + (step_cost if step_cost > 0 else 1)
                new_positions = positions + [new_position]

                # Thêm vào hàng đợi
                if new_cost < visited.get(child, float('inf')):
                    visited[child] = new_cost
                    path[child] = (node, step_cost) 
                    
                    heapq.heappush(que, (new_cost, child, new_positions, new_position))

    return None, 0, ["Không tìm thấy lời giải."]

def DLS(state=None, limit=None, path=None):
    N = BOARD_SIZE
    GOAL_TUPLE = tuple(tuple(row) for row in goal_state)
    
    # Hàm hỗ trợ (Cần được định nghĩa ở phạm vi toàn cục hoặc truyền vào)
    def count_rooks(board_tuple):
        return sum(row.count(1) for row in board_tuple)
    
    # Hàm tái tạo history (Sử dụng logic của IDS/BFS)
    def _reconstruct_history(final_state, final_path, current_limit):
        history = []
        current = final_state
        temp_path = []
        
        while current is not None:
            temp_path.append(current)
            current = final_path.get(current)
        temp_path.reverse()
        
        current_rooks = -1
        for step_board in temp_path:
            num_rooks = count_rooks(step_board)
            if num_rooks > current_rooks:
                # Ghi lịch sử đúng định dạng List of Strings
                history.append(f"--- Đặt quân Xe thứ {num_rooks} ---")
                for r_row in step_board:
                    history.append(f"{list(r_row)}")
                current_rooks = num_rooks
        
        history.append(f"TÌM THẤY LỜI GIẢI BẰNG DLS (Giới hạn: {current_limit})!")
        
        # Trả về [final_board, history] để khớp với cách bạn hứng biến
        return [list(row) for row in final_state], history

    # [Phần khởi tạo]
    if state is None:
        state = tuple(tuple(row) for row in original_state)
        path = {state: None}
        if limit is None:
            limit = N

    # 1. KIỂM TRA ĐÍCH VÀ ĐIỀU KIỆN DỪNG
    if state == GOAL_TUPLE:
        # 💥 SỬA ĐỔI: Tái tạo history và trả về (final_board, history)
        return _reconstruct_history(state, path, limit) 
    elif limit == 0:
        return "cutoff", path
    
    cutoff_occurred = False
    k = count_rooks(state)
    if k == N: 
        return "failure", path
        
    next_row = k
    a = [list(row) for row in state]
    
    # 2. Sinh trạng thái con (Tối ưu hóa theo hàng)
    for j in range(N):
        is_col_free = True
        for r in range(next_row): 
            if a[r][j] == 1:
                is_col_free = False
                break
        
        if is_col_free:
            b = [list(row) for row in a]
            b[next_row][j] = 1
            child = tuple(tuple(row) for row in b)
            
            new_path = path.copy()
            new_path[child] = state
            
            result, final_path = DLS(child, limit - 1, new_path)

            if result == "cutoff":
                cutoff_occurred = True
            elif result != "failure":
                # 💥 SỬA ĐỔI: Nếu result không phải chuỗi lỗi, nó là [final_board, history]
                return result, final_path 
                
    if cutoff_occurred:
        return "cutoff", path
    else:
        return "failure", path
        
def IDS(max_depth=BOARD_SIZE):
    N = BOARD_SIZE
    
    # Hàm đếm quân (phải được định nghĩa ở phạm vi toàn cục hoặc truyền vào)
    def count_rooks(board_tuple):
        return sum(row.count(1) for row in board_tuple)
    
    for depth in range(1, max_depth + 1):
        # Gọi DLS với giới hạn độ sâu tăng dần
        result, history_or_path = DLS(limit=depth) 
        
        # Kiểm tra kiểu dữ liệu: Nếu DLS thành công, nó trả về một LIST [final_board, history]
        if isinstance(result, list): 
            
            # Khai thác kết quả (chúng ta biết result là final_board)
            final_board = result
            history = history_or_path # History được trả về ở vị trí path
            
            # Cập nhật thông báo cuối cùng trong lịch sử (Optional: để khớp với độ sâu)
            history[-1] = history[-1].replace("DLS (Giới hạn:", f"IDS (Độ sâu:")
            
            # TRẢ VỀ: (Final Board, History) - Khớp với cách IDS/BFS trả về
            return final_board, history

        # Nếu thất bại hoặc cắt cụt, tiếp tục vòng lặp

    # Nếu không tìm thấy sau khi đạt max_depth
    return None, ["Không tìm thấy lời giải trong giới hạn độ sâu."]
  
def GreedySearch():
    N = BOARD_SIZE
    
    # Khai báo các biến cần thiết từ phạm vi toàn cục
    GOAL_TUPLE = tuple(tuple(row) for row in goal_state)
    
    # Tạo dictionary {row: col} cho goal_state để tính heuristic
    goal_positions = {}
    for r in range(N):
        try:
            c = goal_state[r].index(1)
            goal_positions[r] = c
        except ValueError:
            pass 
            
    def count_rooks(board_tuple):
        """Đếm số quân xe đã được đặt."""
        return sum(row.count(1) for row in board_tuple)
        
    def heuristic(new_pos, next_row):
        """Tính h(n): Khoảng cách từ vị trí Xe mới đến vị trí Xe đích của hàng đó."""
        r, c = new_pos
        goal_col = goal_positions.get(r, N // 2)
        return abs(c - goal_col)

    original_state_tuple = tuple(tuple(row) for row in original_state)
    
    # que: (h(n), state, positions)
    # Positions: [ (r1, c1), (r2, c2), ... ] - Lưu vết đường đi
    que = []
    heapq.heappush(que, (0, original_state_tuple, [])) 
    visited = set()

    while que:
        h_val, node, positions = heapq.heappop(que)
        
        if node in visited:
            continue
        visited.add(node)
        
        # 1. KIỂM TRA GOAL STATE
        if node == GOAL_TUPLE:
            # --- TÁI TẠO LỊCH SỬ TỪ CHUỖI NƯỚC ĐI (positions) ---
            history = []
            # Bàn cờ hiện tại bắt đầu từ trạng thái rỗng
            current_board = [list(row) for row in original_state]
            
            # Xây dựng lịch sử từng bước dựa trên positions
            for i, (r, c) in enumerate(positions):
                # 💥 Tái tạo lại ma trận từ đầu để đảm bảo tính độc lập
                # (Sử dụng các nước đi tích lũy)
                current_board[r][c] = 1 
                num_rooks = i + 1
                
                # Ghi lịch sử đúng định dạng List of Strings
                history.append(f"--- Đặt quân Xe thứ {num_rooks} (h(n) cuối: {h_val}) ---")
                for r_row in current_board:
                    history.append(f"{r_row}")
            
            history.append("TÌM THẤY LỜI GIẢI HỢP LỆ!")
            
            # TRẢ VỀ: (Final Board, History)
            return [list(row) for row in node], history

        # 2. Sinh trạng thái con (Tối ưu hóa theo hàng)
        k = count_rooks(node)
        if k == N:
            continue

        next_row = k
        a = [list(row) for row in node]
        
        for j in range(N):
            # Kiểm tra cột có trống không (tính hợp lệ)
            is_col_free = True
            for r in range(next_row): 
                if a[r][j] == 1:
                    is_col_free = False
                    break
            
            if is_col_free:
                # Tạo trạng thái con
                b = [list(row) for row in a]
                b[next_row][j] = 1
                child = tuple(tuple(row) for row in b)
                
                # Tính h(n) cho trạng thái con
                new_pos = (next_row, j)
                h_n = heuristic(new_pos, next_row)
                
                new_positions = positions + [new_pos]
                
                # Chỉ thêm vào que nếu chưa được visited
                if child not in visited: 
                    heapq.heappush(que, (h_n, child, new_positions))

    return None, ["Không tìm thấy lời giải."]
    
def A_star():
    N = BOARD_SIZE
    
    # Khai báo các biến cần thiết từ phạm vi toàn cục
    GOAL_TUPLE = tuple(tuple(row) for row in goal_state)
    GOAL_POS_LAST = (N - 1, N - 1)
    
    def count_rooks(board_tuple):
        """Hàm hỗ trợ đếm số quân xe."""
        return sum(row.count(1) for row in board_tuple)
        
    def distance(p1, p2):
        """g(n) và h(n) đều dùng Manhattan distance."""
        return abs(p1[0] - p2[0]) + abs(p1[1] - p2[1])

    original_state_tuple = tuple(tuple(row) for row in original_state)
    que = []
    # (f(n), g(n), state, positions, parent_node, step_cost)
    heapq.heappush(que, (0, 0, original_state_tuple, [], None, 0)) 
    
    # visited: {state_tuple: min_g_val}
    visited = {original_state_tuple: 0}
    # path: {child_state: (parent_state, step_cost)}
    path = {original_state_tuple: (None, 0)}

    while que:
        f_val, g_val, node, positions, _, _ = heapq.heappop(que)
        
        # 1. Kiểm tra Visited/Relaxation
        if g_val > visited.get(node, float('inf')):
             continue

        # 2. KIỂM TRA GOAL STATE
        if node == GOAL_TUPLE:
            # --- TÁI TẠO LỊCH SỬ VÀ CHI PHÍ ---
            final_board = [list(row) for row in node]
            history = []
            current = node
            temp_path = []
            
            while current is not None:
                parent, step_cost = path.get(current, (None, 0))
                temp_path.append((current, step_cost))
                current = parent
            
            temp_path.reverse()
            
            current_rooks = -1
            total_cost = 0
            for step_board, step_cost in temp_path:
                num_rooks = count_rooks(step_board)
                
                if num_rooks > current_rooks:
                    total_cost += step_cost
                    history.append(f"--- Đặt quân Xe thứ {num_rooks} (Chi phí: {step_cost}) ---")
                    
                    for r_row in step_board:
                        history.append(f"{list(r_row)}")
                    
                    current_rooks = num_rooks
            
            history.append(f"TÌM THẤY LỜI GIẢI HỢP LỆ! TỔNG CHI PHÍ: {total_cost}")
            
            # TRẢ VỀ: (Final Board, Total Cost, History)
            return final_board, total_cost, history

        # 3. TỐI ƯU HÓA: Chỉ sinh trạng thái hợp lệ bằng cách đặt Xe theo hàng
        k = count_rooks(node)
        if k == N:
            continue

        next_row = k
        a = [list(row) for row in node]
        
        # 💥 SỬA LỖI HIỆU SUẤT: Loại bỏ vòng lặp i lồng ngoài và chỉ duyệt cột j
        i = next_row # Hàng hiện tại được xác định bởi k
        
        for j in range(N):
            
            # Kiểm tra cột trống (bắt buộc cho N Xe)
            is_col_free = True
            for r in range(i): 
                if a[r][j] == 1:
                    is_col_free = False
                    break
            
            if not is_col_free:
                continue

            # Sinh trạng thái con
            b = [list(row) for row in a]
            b[i][j] = 1
            child = tuple(tuple(row) for row in b)
            
            new_position = (i, j)
            
            # Tính g(n) và f(n) mới
            if not positions:
                step_cost = distance((0, 0), new_position)
            else:
                step_cost = distance(positions[-1], new_position)
                
            g_new = g_val + (step_cost if step_cost > 0 else 1)

            # h(n): ước lượng còn lại (dùng Manhattan đến góc cuối)
            h_new = distance(new_position, GOAL_POS_LAST)
            f_new = g_new + h_new
            
            new_positions = positions + [new_position]

            # 4. Kiểm tra và thêm vào hàng đợi (Relaxation)
            if g_new < visited.get(child, float('inf')):
                visited[child] = g_new
                path[child] = (node, step_cost) 
                
                # Đẩy vào que: (f_new, g_new, state, positions, parent, step_cost)
                heapq.heappush(que, (f_new, g_new, child, new_positions, node, step_cost))

    return None, 0, ["Không tìm thấy lời giải."]
    
def Hill_Climbing(goal_state=goal_state, BOARD_SIZE=BOARD_SIZE):
    def conflicts(state):
        cnt = 0
        for i in range(BOARD_SIZE):
            for j in range(BOARD_SIZE):
                if state[i][j] == 1:
                    cnt += sum(state[i][k] for k in range(BOARD_SIZE)) - 1
                    cnt += sum(state[k][j] for k in range(BOARD_SIZE)) - 1
        return cnt // 2

    def random_state():
        state = [[0]*BOARD_SIZE for _ in range(BOARD_SIZE)]
        for i in range(BOARD_SIZE):
            j = random.randint(0, BOARD_SIZE-1)
            state[i][j] = 1
        return state

    current = random_state()
    current_cost = conflicts(current)
    history = [copy.deepcopy(current)]  # Lưu lại bước đầu tiên

    while True:
        neighbors = []
        for i in range(BOARD_SIZE):
            for j in range(BOARD_SIZE):
                if current[i][j] == 1:
                    for new_j in range(BOARD_SIZE):
                        if new_j != j:
                            new_state = [row[:] for row in current]
                            new_state[i][j] = 0
                            new_state[i][new_j] = 1
                            neighbors.append(new_state)

        next_state = None
        next_cost = float("inf")
        for n in neighbors:
            c = conflicts(n)
            if c < next_cost:
                next_cost = c
                next_state = n

        if next_cost >= current_cost:  # Local minima
            return current, history

        current = next_state
        current_cost = next_cost
        history.append(copy.deepcopy(current))

        if current == goal_state or current_cost == 0:
            return current, history

def Simulated_Annealing(goal_state=goal_state, BOARD_SIZE=BOARD_SIZE, max_iter=100000, T=100, cooling=0.99):
    def conflicts(state):
        cnt = 0
        for i in range(BOARD_SIZE):
            for j in range(BOARD_SIZE):
                if state[i][j] == 1:
                    cnt += sum(state[i][k] for k in range(BOARD_SIZE)) - 1
                    cnt += sum(state[k][j] for k in range(BOARD_SIZE)) - 1
        return cnt // 2

    def random_state():
        state = [[0]*BOARD_SIZE for _ in range(BOARD_SIZE)]
        for i in range(BOARD_SIZE):
            j = random.randint(0, BOARD_SIZE-1)
            state[i][j] = 1
        return state

    def random_neighbor(state):
        neighbor = [row[:] for row in state]
        row = random.randint(0, BOARD_SIZE-1)
        col_old = neighbor[row].index(1)
        col_new = random.randint(0, BOARD_SIZE-1)
        while col_new == col_old:
            col_new = random.randint(0, BOARD_SIZE-1)
        neighbor[row][col_old] = 0
        neighbor[row][col_new] = 1
        return neighbor

    current = random_state()
    current_cost = conflicts(current)
    history = [current]

    for step in range(max_iter):
        if current_cost == 0 or (goal_state is not None and current == goal_state):
            return current, history

        neighbor = random_neighbor(current)
        neighbor_cost = conflicts(neighbor)
        delta = neighbor_cost - current_cost

        if delta < 0 or random.random() < math.exp(-delta / T):
            current, current_cost = neighbor, neighbor_cost
            history.append(current)

        T *= cooling
        if T < 1e-6:
            break

    return current, history

def Genetic_Algorithm(target=goal_state, N=BOARD_SIZE, pop_size=100, mutation_rate=0.1, max_gen=1000):
    history = []  # lưu các cha mẹ từng thế hệ + con đích

    def random_state():
        state = [[0] * N for _ in range(N)]
        for i in range(N):
            c = random.randint(0, N - 1)
            state[i][c] = 1
        return state

    def fitness(state):
        cost = 0
        for i in range(N):
            col_state = state[i].index(1)
            col_target = target[i].index(1)
            cost += abs(col_state - col_target)
        return cost

    def selection(population):
        total_fit = sum(1 / (1 + fitness(ind)) for ind in population)
        pick = random.uniform(0, total_fit)
        current = 0
        for ind in population:
            current += 1 / (1 + fitness(ind))
            if current > pick:
                return ind
        return population[-1]

    def crossover(p1, p2):
        point = random.randint(1, N - 2)
        child1 = [row[:] for row in p1[:point]] + [row[:] for row in p2[point:]]
        child2 = [row[:] for row in p2[:point]] + [row[:] for row in p1[point:]]
        return child1, child2

    def mutate(state):
        if random.random() < mutation_rate:
            r = random.randint(0, N - 1)
            c_old = state[r].index(1)
            c_new = random.randint(0, N - 1)
            while c_new == c_old:
                c_new = random.randint(0, N - 1)
            state[r][c_old] = 0
            state[r][c_new] = 1
        return state

    # --- main loop ---
    population = [random_state() for _ in range(pop_size)]
    best = min(population, key=fitness)

    for _ in range(max_gen):
        if best == target:
            history.append(best)  # thêm con đích vào lịch sử
            return best, history

        new_population = []
        parents_this_gen = []  # lưu cha mẹ của thế hệ này
        while len(new_population) < pop_size:
            p1 = selection(population)
            p2 = selection(population)
            parents_this_gen.extend([p1, p2])
            c1, c2 = crossover(p1, p2)
            new_population.append(mutate(c1))
            new_population.append(mutate(c2))

        history.append(parents_this_gen)  # lưu cha mẹ của thế hệ
        population = new_population
        best = min(population, key=fitness)

    history.append(best)  # nếu kết thúc mà chưa đạt target, lưu best
    return best, history

def Beam_Search(k=2):
    # ----- TẤT CẢ LOGIC TÌM KIẾM CỦA BẠN ĐƯỢC GIỮ NGUYÊN -----
    history_state = []  # Vẫn lưu dạng [1, 3, 0] như cũ

    def cost_function(state):
        conflicts = 0
        for i in range(len(state)):
            for j in range(i + 1, len(state)):
                if state[i] == state[j] or abs(state[i] - state[j]) == j - i:
                    conflicts += 1
        return conflicts

    beam = [(0, [])]

    for row in range(BOARD_SIZE):
        candidates = []
        for cost, state in beam:
            for col in range(BOARD_SIZE):
                if col not in state:
                    new_state = state + [col]
                    new_cost = cost_function(new_state)
                    candidates.append((new_cost, new_state))

        if not candidates:
            # --- THAY ĐỔI 1: Định dạng lại history_state trước khi return ---
            reformatted_history = []
            for state in history_state:
                board_step = [[0] * BOARD_SIZE for _ in range(BOARD_SIZE)]
                for r, c in enumerate(state):
                    board_step[r][c] = 1
                reformatted_history.append(board_step)
            return None, reformatted_history

        beam = heapq.nsmallest(k, candidates, key=lambda x: x[0])
        best_cost, best_state = min(beam, key=lambda x: x[0])
        history_state.append(best_state) # Vẫn append state [1, 3, 0] như cũ

        if len(best_state) == BOARD_SIZE and best_cost == 0:
            board = [[0] * BOARD_SIZE for _ in range(BOARD_SIZE)]
            for r, c in enumerate(best_state):
                board[r][c] = 1
            
            # --- THAY ĐỔI 2: Định dạng lại history_state trước khi return ---
            reformatted_history = []
            for state in history_state:
                board_step = [[0] * BOARD_SIZE for _ in range(BOARD_SIZE)]
                for r, c in enumerate(state):
                    board_step[r][c] = 1
                reformatted_history.append(board_step)
            return board, reformatted_history

    # ----- LOGIC KẾT THÚC CỦA BẠN ĐƯỢC GIỮ NGUYÊN -----
    best_cost, best_state = min(beam, key=lambda x: x[0])
    board = [[0] * BOARD_SIZE for _ in range(BOARD_SIZE)]
    for r, c in enumerate(best_state):
        board[r][c] = 1

    # --- THAY ĐỔI 3: Định dạng lại history_state lần cuối trước khi return ---
    reformatted_history = []
    for state in history_state:
        board_step = [[0] * BOARD_SIZE for _ in range(BOARD_SIZE)]
        for r, c in enumerate(state):
            board_step[r][c] = 1
        reformatted_history.append(board_step)
        
    return board, reformatted_history
    
def DFS_AND_OR():
    N = BOARD_SIZE
    goal_state_tuple = tuple(tuple(row) for row in goal_state)
    
    def count_rooks(state_tuple):
        """Hàm hỗ trợ đếm số quân xe."""
        return sum(sum(row) for row in state_tuple)

    # --- Hàm đệ quy nội tuyến thực hiện logic tìm kiếm cốt lõi ---
    def _search(state_tuple, path_history):
        if state_tuple == goal_state_tuple:
            return [state_tuple]

        if state_tuple in path_history: 
            return None

        # TỐI ƯU HÓA: Xác định hàng tiếp theo để đặt quân
        k = count_rooks(state_tuple)
        if k == N: 
            return None 

        next_row = k 
        current_board = [list(row) for row in state_tuple]

        # Logic OR: Duyệt qua các cột chỉ trong HÀNG TIẾP THEO
        r = next_row 
        
        for c in range(N):
            
            # Kiểm tra tính hợp lệ nhanh (trùng cột)
            is_col_free = True
            for r_prev in range(r):
                if current_board[r_prev][c] == 1:
                    is_col_free = False
                    break
            
            if is_col_free:
                # Sinh trạng thái con
                temp_board = [row[:] for row in current_board]
                temp_board[r][c] = 1 
                result_state = tuple(tuple(row) for row in temp_board)
                
                # Logic AND/OR: Gọi đệ quy
                path_from_child = _search(result_state, path_history + [state_tuple])
                
                if path_from_child is not None:
                    # Lời giải được tìm thấy
                    full_path = [state_tuple] + path_from_child
                    return full_path
        
        return None

    # --- Phần thân chính của hàm ---
    initial_state_tuple = tuple(tuple(row) for row in original_state)
    final_path_tuples = _search(initial_state_tuple, [])
    
    if final_path_tuples is not None:
        
        final_board = [list(row) for row in final_path_tuples[-1]]
        
        # 2. TÁI TẠO LỊCH SỬ DƯỚI DẠNG LIST CÁC CHUỖI KHÔNG CHỨA "\n"
        history_log = ["===== CHUỖI NƯỚC ĐI DFS-AND/OR TỐI ƯU ====="]
        last_k = -1
        
        for state in final_path_tuples:
            k = count_rooks(state)
            
            # Chỉ ghi lại lịch sử khi có quân mới được đặt
            if k > last_k:
                history_log.append("-" * 30)
                history_log.append(f"Lượt {k} / Đặt Xe thứ {k}:")
                
                # Ghi TỪNG HÀNG của ma trận thành MỘT MỤC RIÊNG trong list
                for row in state:
                    history_log.append(f"{list(row)}") # Đảm bảo là list string
                
                last_k = k
        
        history_log.append("-" * 30)
        history_log.append("TÌM THẤY LỜI GIẢI HỢP LỆ.")
        
        # Trả về final_board và history_log (list các chuỗi)
        return final_board, history_log
    else:
        # Trả về None và một list rỗng nếu không tìm thấy
        return None, ["Không tìm thấy lời giải hợp lệ."]

def Belief_Search(N=BOARD_SIZE):
    def valid(state):
        # Kiểm tra không có 2 xe cùng cột
        for c in range(N):
            if sum(state[r][c] for r in range(N)) > 1:
                return False
        return True

    def count_rooks(state):
        return sum(sum(row) for row in state)

    # --- Trạng thái niềm tin ban đầu ---
    s1 = [[0]*N for _ in range(N)]
    s2 = [[0]*N for _ in range(N)]
    s1[0][0], s1[1][2] = 1, 1
    s2[0][1], s2[1][3], s2[2][0] = 1, 1, 1
    belief_set = [s1, s2]

    history = "===== TẬP TRẠNG THÁI BAN ĐẦU =====\n"
    for i, s in enumerate(belief_set, 1):
        history += f"\n--- State {i} ({count_rooks(s)} xe) ---\n"
        for row in s:
            history += f"{row}\n"

    step = 1
    while True:
        # Dừng nếu tất cả state đã đủ N xe
        if all(count_rooks(s) == N for s in belief_set):
            history += "\nTất cả bàn cờ đã đủ 4 xe!\n"
            break

        history += f"\n\n===== TẬP TRẠNG THÁI BƯỚC {step} =====\n"
        new_set = []

        # --- Sinh tập trạng thái mới ---
        for idx, s in enumerate(belief_set):
            if count_rooks(s) < N:
                empty_rows = [r for r in range(N) if sum(s[r]) == 0]
                if empty_rows:
                    r = random.choice(empty_rows)
                    history += f"\nTừ State {idx+1} (chọn hàng {r+1} để sinh 4 state mới):\n"
                    for c in range(N):
                        new_state = [row[:] for row in s]
                        new_state[r][c] = 1
                        history += f"\nState mới ({r},{c}):\n"
                        for row in new_state:
                            history += f"{row}\n"
                        new_set.append(new_state)
                else:
                    new_set.append(s)
            else:
                new_set.append(s)

        # --- Kiểm tra hợp lệ ---
        valid_set = []
        history += f"\n===== ĐÁNH GIÁ SAU BƯỚC {step} =====\n"
        for i, s in enumerate(new_set, 1):
            if valid(s):
                valid_set.append(s)
                history += f"\nState {i} hợp lệ:\n"
                for row in s:
                    history += f"{row}\n"
            else:
                history += f"\nState {i} phạm luật (trùng cột):\n"
                for row in s:
                    history += f"{row}\n"

        belief_set = valid_set
        if not belief_set:
            history += "\nTất cả state bị loại → không tìm được lời giải.\n"
            return None, history

        step += 1

    # --- Kết quả cuối ---
    final = [s for s in belief_set if valid(s)]
    if final:
        history += "\nTÌM ĐƯỢC LỜI GIẢI HỢP LỆ!\n"
        for row in final[0]:
            history += f"{row}\n"
        return final[0], history
    else:
        history += "\nKhông có bàn cờ hợp lệ trong tập cuối cùng.\n"
        return None, history

def Partial_Observable_Search(N=BOARD_SIZE):
    def valid(state):
        # Kiểm tra không có 2 xe cùng hàng hoặc cùng cột
        for i in range(N):
            if sum(state[i]) > 1:  # cùng hàng
                return False
            if sum(state[r][i] for r in range(N)) > 1:  # cùng cột
                return False
        return True

    def count_rooks(state):
        return sum(sum(row) for row in state)

    # --- Trạng thái quan sát được 1 phần ---
    # Chỉ biết vị trí của 1 vài xe (giống như chỉ thấy 1 phần bàn cờ)
    observed_state = [[0]*N for _ in range(N)]
    observed_state[0][1] = 1  # quan sát thấy 1 xe ở (0,1)
    observed_state[2][3] = 1  # và 1 xe ở (2,3)

    # --- Tập trạng thái khả dĩ (niềm tin) ---
    belief_set = []
    for _ in range(3):  # sinh 3 trạng thái khả dĩ khác nhau
        s = [row[:] for row in observed_state]
        # thêm ngẫu nhiên 1-2 xe ở vị trí chưa biết
        while count_rooks(s) < random.randint(2, 3):
            r, c = random.randint(0, N-1), random.randint(0, N-1)
            if s[r][c] == 0:
                s[r][c] = 1
        belief_set.append(s)

    history = "===== TRẠNG THÁI QUAN SÁT 1 PHẦN =====\n"
    for i, s in enumerate(belief_set, 1):
        history += f"\n--- State {i} ---\n"
        for row in s:
            history += f"{row}\n"

    step = 1
    while True:
        if all(count_rooks(s) == N for s in belief_set):
            history += "\nMọi bàn cờ khả dĩ đều đủ 4 xe!\n"
            break

        history += f"\n\n===== BƯỚC {step}: CẬP NHẬT NIỀM TIN =====\n"
        new_belief = []

        for idx, s in enumerate(belief_set):
            if count_rooks(s) < N:
                # chọn ngẫu nhiên 1 hàng chưa có xe để thử đặt thêm
                empty_rows = [r for r in range(N) if sum(s[r]) == 0]
                if empty_rows:
                    r = random.choice(empty_rows)
                    for c in range(N):
                        new_state = [row[:] for row in s]
                        new_state[r][c] = 1
                        new_belief.append(new_state)
                        history += f"\nTừ State {idx+1} sinh ({r},{c}):\n"
                        for row in new_state:
                            history += f"{row}\n"

        # lọc trạng thái hợp lệ
        valid_set = [s for s in new_belief if valid(s)]
        history += f"\n===== ĐÁNH GIÁ SAU BƯỚC {step} =====\n"
        for i, s in enumerate(valid_set, 1):
            history += f"\nState {i} hợp lệ:\n"
            for row in s:
                history += f"{row}\n"

        if not valid_set:
            history += "\nTất cả state bị loại → không còn khả năng quan sát hợp lệ.\n"
            return None, history

        belief_set = valid_set
        step += 1

    final = [s for s in belief_set if valid(s)]
    if final:
        history += "\nTÌM ĐƯỢC TẬP TRẠNG THÁI HỢP LỆ!\n"
        for row in final[0]:
            history += f"{row}\n"
        return final[0], history
    else:
        history += "\nKhông có trạng thái hợp lệ.\n"
        return None, history

def Backtrack_Search(N=BOARD_SIZE):
    variables = list(range(N))  # tập biến: các hàng
    domains = {v: list(range(N)) for v in variables}  # tập giá trị: các cột khả dĩ

    history = "===== CSP BACKTRACKING =====\n"
    history_log = [history]
    history_log.append(f"Tập biến (Rows) = {variables}\n")
    history_log.append(f"Tập giá trị (Columns) = {[i for i in range(N)]}\n\n")

    def is_consistent(assignment, var, value):
        for r, c in assignment.items():
            if c == value:  # trùng cột
                return False
        return True

    def print_state(assignment):
        board = [[0]*N for _ in range(N)]
        for r, c in assignment.items():
            board[r][c] = 1
        history_log.append("State:\n")
        for row in board:
            history_log.append(f"{row}\n")
        history_log.append("\n")

    def backtrack(assignment):
        if len(assignment) == N:
            return assignment

        # chọn hàng chưa gán ngẫu nhiên
        remaining_vars = [v for v in variables if v not in assignment]
        var = random.choice(remaining_vars)

        # chọn cột khả dĩ ngẫu nhiên
        possible_values = [v for v in domains[var] if is_consistent(assignment, var, v)]
        random.shuffle(possible_values)
        for value in possible_values:
            assignment[var] = value
            print_state(assignment)  # lưu state sau khi đặt quân
            result = backtrack(assignment)
            if result is not None:
                return result
            del assignment[var]  # quay lui
        return None

    result = backtrack({})

    if result:
        final_board = [[0]*N for _ in range(N)]
        for r, c in result.items():
            final_board[r][c] = 1
        history_log.append("TÌM ĐƯỢC LỜI GIẢI HỢP LỆ!\n")
        for row in final_board:
            history_log.append(f"{row}\n")
        return final_board, "".join(history_log)
    else:
        history_log.append("Không có lời giải.\n")
        return None, "".join(history_log)
        
def Forward_Checking_Search(N=BOARD_SIZE):
    variables = list(range(N))  # tập biến: các hàng
    domains = {v: list(range(N)) for v in variables}  # tập giá trị: các cột khả dĩ

    history = "===== CSP FORWARD CHECKING =====\n"
    history_log = [history]
    history_log.append(f"Tập biến (Rows) = {variables}\n")
    history_log.append(f"Tập giá trị (Columns) = {[i for i in range(N)]}\n\n")

    def is_consistent(assignment, var, value):
        for r, c in assignment.items():
            if c == value:  # trùng cột
                return False
        return True

    def print_state(assignment):
        board = [[0]*N for _ in range(N)]
        for r, c in assignment.items():
            board[r][c] = 1
        history_log.append("State:\n")
        for row in board:
            history_log.append(f"{row}\n")
        history_log.append("\n")

    def backtrack(assignment, local_domains):
        if len(assignment) == N:
            return assignment

        remaining_vars = [v for v in variables if v not in assignment]
        var = random.choice(remaining_vars)

        # chọn các giá trị hợp lệ theo miền hiện tại
        possible_values = [v for v in local_domains[var] if is_consistent(assignment, var, v)]
        random.shuffle(possible_values)

        for value in possible_values:
            assignment[var] = value
            print_state(assignment)

            # forward checking: cập nhật miền biến chưa gán
            new_domains = copy.deepcopy(local_domains)
            del new_domains[var]
            for other_var in new_domains:
                new_domains[other_var] = [v for v in new_domains[other_var] if v != value]

            # nếu có biến nào mất hết giá trị khả dĩ → quay lui ngay
            if any(len(vals) == 0 for vals in new_domains.values()):
                del assignment[var]
                continue

            result = backtrack(assignment, new_domains)
            if result is not None:
                return result

            # quay lui
            del assignment[var]

        return None

    result = backtrack({}, domains)

    if result:
        final_board = [[0]*N for _ in range(N)]
        for r, c in result.items():
            final_board[r][c] = 1
        history_log.append("TÌM ĐƯỢC LỜI GIẢI HỢP LỆ!\n")
        for row in final_board:
            history_log.append(f"{row}\n")
        return final_board, "".join(history_log)
    else:
        history_log.append("Không có lời giải.\n")
        return None, "".join(history_log)
        
def AC3_Search(N=BOARD_SIZE):
    variables = list(range(N))  # tập biến: các hàng
    domains = {v: list(range(N)) for v in variables}  # tập giá trị: các cột khả dĩ

    history_log = []
    history_log.append("===== CSP AC-3 =====\n")
    history_log.append(f"Tập biến (Rows) = {variables}\n")
    history_log.append(f"Tập giá trị (Columns) = {[i for i in range(N)]}\n\n")

    # Kiểm tra tính tương thích của 2 biến
    def is_consistent(var1, val1, var2, val2):
        return val1 != val2  # không trùng cột

    # AC-3: lọc miền
    def ac3(domains):
        queue = deque([(Xi, Xj) for Xi in variables for Xj in variables if Xi != Xj])
        while queue:
            Xi, Xj = queue.popleft()
            if revise(Xi, Xj, domains):
                if len(domains[Xi]) == 0:
                    return False  # miền rỗng, không có giải
                for Xk in variables:
                    if Xk != Xi and Xk != Xj:
                        queue.append((Xk, Xi))
        return True

    def revise(Xi, Xj, domains):
        revised = False
        for x in domains[Xi][:]:
            if all(not is_consistent(Xi, x, Xj, y) for y in domains[Xj]):
                domains[Xi].remove(x)
                revised = True
        return revised

    # In trạng thái miền hiện tại
    def print_state(domains):
        history_log.append("State (Domains):\n")
        for var in variables:
            history_log.append(f"Row {var}: {domains[var]}\n")
        history_log.append("\n")

    # Backtracking kết hợp AC-3
    def backtrack(assignment, local_domains):
        if len(assignment) == N:
            return assignment

        remaining_vars = [v for v in variables if v not in assignment]
        var = random.choice(remaining_vars)

        # random giá trị trong miền hiện tại
        possible_values = local_domains[var][:]
        random.shuffle(possible_values)

        for value in possible_values:
            assignment[var] = value
            new_domains = copy.deepcopy(local_domains)
            new_domains[var] = [value]

            # lọc miền bằng AC-3
            if ac3(new_domains):
                print_state(new_domains)
                result = backtrack(assignment, new_domains)
                if result is not None:
                    return result

            # quay lui
            del assignment[var]

        return None

    result = backtrack({}, domains)

    if result:
        final_board = [[0]*N for _ in range(N)]
        for r, c in result.items():
            final_board[r][c] = 1
        history_log.append("TÌM ĐƯỢC LỜI GIẢI HỢP LỆ!\n")
        for row in final_board:
            history_log.append(f"{row}\n")
        return final_board, "".join(history_log)
    else:
        history_log.append("Không có lời giải.\n")
        return None, "".join(history_log)       
        

def Minimax_Search(N=BOARD_SIZE):
    history_log = [f"==== ADVERSARIAL MINIMAX (WEIGHTED {N}x{N} BOARD) ====\n"]
    
    # 1. KHỞI TẠO CỤC BỘ (Để đảm bảo các biến được tìm thấy)
    WEIGHT_MATRIX = [[random.randint(1, 10) for _ in range(N)] for _ in range(N)]
    history_log.append("Ma trận Trọng số (Giá trị mỗi ô):\n")
    history_log.append("\n".join([f"{row}" for row in WEIGHT_MATRIX]) + "\n")

    def render_board(matrix):
        # Hàm phụ trợ để in ra ma trận 0/1 hoặc ma trận trọng số
        return "\n".join([f"{row}" for row in matrix])

    def evaluate(state):
        max_score = 0
        min_score = 0
        sorted_items = sorted(state.items())
        
        for i, (r, c) in enumerate(sorted_items):
             # Phân biệt quân MAX/MIN theo lượt đặt
            if i % 2 == 0: 
                max_score += WEIGHT_MATRIX[r][c]
            else: 
                min_score += WEIGHT_MATRIX[r][c]

        return max_score - min_score

    def get_valid_moves(current_state):
        valid_moves = []
        assigned_rows = current_state.keys()
        assigned_cols = current_state.values()
        remaining_rows = [r for r in range(N) if r not in assigned_rows]
        if not remaining_rows: return []
        next_row = min(remaining_rows)
        for col in range(N):
            if col not in assigned_cols:
                valid_moves.append((next_row, col))
        return valid_moves

    def apply_move(current_state, move):
        r, c = move
        new_state = current_state.copy()
        new_state[r] = c
        return new_state

    # 2. HÀM MINIMAX (Tìm kiếm đường đi tối ưu)
    def minimax(current_state, depth, is_maximizing_player):
        if depth == 0 or len(current_state) == N:
            return evaluate(current_state), None, [current_state] 

        best_eval = -float('inf') if is_maximizing_player else float('inf')
        best_move = None
        best_path = []
        
        possible_moves = get_valid_moves(current_state)
        
        # Tối ưu hóa: Sắp xếp nước đi (giúp tăng tốc Alpha-Beta nếu có)
        possible_moves.sort(key=lambda move: WEIGHT_MATRIX[move[0]][move[1]], reverse=True)
        
        for move in possible_moves:
            new_state = apply_move(current_state, move)
            # Không cần ghi log đệ quy, chỉ cần tìm đường đi
            eval_score, _, path_from_child = minimax(new_state, depth - 1, not is_maximizing_player)
            
            if (is_maximizing_player and eval_score > best_eval) or \
               (not is_maximizing_player and eval_score < best_eval):
                    best_eval = eval_score
                    best_move = move
                    best_path = [new_state] + path_from_child
        
        return best_eval, best_move, best_path

    # 3. PHẦN THÂN CHÍNH (Tạo lịch sử đơn giản từ best_path)
    initial_assignment = {} 
    max_depth = N 

    score, best_move, best_path = minimax(initial_assignment, max_depth, True)
    
    # Xử lý kết quả và tạo lịch sử đơn giản
    if best_move:
        history_log.append("\n============= LỊCH SỬ CHUỖI NƯỚC ĐI TỐI ƯU =============\n")
        
        # Tạo lịch sử hành động từ chuỗi trạng thái tối ưu
        current_state_temp = initial_assignment.copy()
        
        for i, move_assignment in enumerate(best_path):
            
            # Tìm nước đi mới được thêm vào
            r_new = list(set(move_assignment.keys()) - set(current_state_temp.keys()))
            if not r_new: continue # Bỏ qua nếu không tìm thấy sự khác biệt (xảy ra ở bước đầu tiên)

            r_new = r_new[0]
            c_new = move_assignment[r_new]
            move = (r_new, c_new)
                 
            player = "MAX" if i % 2 == 0 else "MIN"
            weight = WEIGHT_MATRIX[r_new][c_new]

            action = "TỐI ĐA hóa điểm số cho tôi" if player == "MAX" else "TỐI THIỂU hóa điểm số của đối thủ"
            
            # Ghi lịch sử đơn giản
            history_log.append(f"Lượt {i+1} ({player}): Chọn ô {move} (Trọng số: {weight}) vì {action}.\n")
            
            # Cập nhật trạng thái hiện tại
            current_state_temp = move_assignment
            
        final_assignment = best_path[-1]  
        final_board = [[0]*N for _ in range(N)]
        for r, c in final_assignment.items():
            final_board[r][c] = 1
             
        history_log.append("\n============= KẾT LUẬN =====================\n")
        history_log.append(f"Điểm Chênh Lệch Cuối Cùng (MAX - MIN): {score}\n")
        history_log.append("BÀN CỜ KHI HOÀN TẤT CHUỖI NƯỚC ĐI TỐI ƯU:\n")
        history_log.append(render_board(final_board))
        
        return final_board, "".join(history_log)
    else:
        history_log.append("Không có nước đi khả dĩ.\n")
        return None, "".join(history_log)
        
def Alpha_Beta_Pruning_Search(N=BOARD_SIZE):
    history_log = [f"===== ADVERSARIAL ALPHA-BETA PRUNING (WEIGHTED {N}x{N} BOARD) =====\n"]
    
    # THÔNG BÁO VỀ KHÁC BIỆT ALPHA-BETA VÀ MINIMAX (Sự khác biệt về Hiệu suất)
    history_log.append("✅ Thuật toán này cho kết quả giống Minimax nhưng **nhanh hơn** vì đã áp dụng kỹ thuật Tỉa (Pruning).")
    history_log.append("Alpha-Beta đã **bỏ qua việc đánh giá** các nhánh (nước đi) mà không thể dẫn đến kết quả tối ưu cuối cùng.\n")
    
    # 1. KHỞI TẠO CỤC BỘ VÀ HÀM PHỤ (Giống Minimax)
    WEIGHT_MATRIX = [[random.randint(1, 10) for _ in range(N)] for _ in range(N)]
    history_log.append("Ma trận Trọng số (Giá trị mỗi ô):\n")
    history_log.append("\n".join([f"{row}" for row in WEIGHT_MATRIX]) + "\n")

    def render_board(matrix):
        return "\n".join([f"{row}" for row in matrix])

    def evaluate(state):
        max_score = 0
        min_score = 0
        sorted_items = sorted(state.items())
        
        for i, (r, c) in enumerate(sorted_items):
            if i % 2 == 0: 
                max_score += WEIGHT_MATRIX[r][c]
            else: 
                min_score += WEIGHT_MATRIX[r][c]
        return max_score - min_score

    def get_valid_moves(current_state):
        valid_moves = []
        assigned_rows = current_state.keys()
        assigned_cols = current_state.values()
        remaining_rows = [r for r in range(N) if r not in assigned_rows]
        if not remaining_rows: return []
        next_row = min(remaining_rows)
        for col in range(N):
            if col not in assigned_cols:
                valid_moves.append((next_row, col))
        return valid_moves

    def apply_move(current_state, move):
        r, c = move
        new_state = current_state.copy()
        new_state[r] = c
        return new_state

    # 2. HÀM ALPHA-BETA PRUNING (Có logic tỉa Alpha và Beta)
    def alpha_beta(current_state, depth, alpha, beta, is_maximizing_player):
        if depth == 0 or len(current_state) == N:
            return evaluate(current_state), None, [current_state]

        best_move = None
        best_path = []
        possible_moves = get_valid_moves(current_state)
        possible_moves.sort(key=lambda move: WEIGHT_MATRIX[move[0]][move[1]], reverse=True)

        if is_maximizing_player:
            max_eval = -float('inf')
            
            for move in possible_moves:
                new_state = apply_move(current_state, move)
                eval_score, _, path_from_child = alpha_beta(new_state, depth - 1, alpha, beta, False)
                
                if eval_score > max_eval:
                    max_eval = eval_score
                    best_move = move
                    best_path = [new_state] + path_from_child

                alpha = max(alpha, max_eval)
                
                # Tỉa Beta (Beta Pruning) - Bỏ qua nhánh không cần thiết
                if beta <= alpha:
                    break 
            
            return max_eval, best_move, best_path
        
        else: # Minimizing Player
            min_eval = float('inf')

            for move in possible_moves:
                new_state = apply_move(current_state, move)
                eval_score, _, path_from_child = alpha_beta(new_state, depth - 1, alpha, beta, True)
                
                if eval_score < min_eval:
                    min_eval = eval_score
                    best_move = move
                    best_path = [new_state] + path_from_child

                beta = min(beta, min_eval)

                # Tỉa Alpha (Alpha Pruning) - Bỏ qua nhánh không cần thiết
                if beta <= alpha:
                    break 
            
            return min_eval, best_move, best_path

    # 3. PHẦN THÂN CHÍNH
    initial_assignment = {} 
    max_depth = N 

    score, best_move, best_path = alpha_beta(initial_assignment, max_depth, -float('inf'), float('inf'), True)
    
    
    # Xử lý kết quả và tạo lịch sử đơn giản
    if best_move:
        history_log.append("\n================ LỊCH SỬ CHUỖI NƯỚC ĐI TỐI ƯU ================\n")
        
        current_state_temp = initial_assignment.copy()
        
        for i, move_assignment in enumerate(best_path):
            r_new = list(set(move_assignment.keys()) - set(current_state_temp.keys()))
            if not r_new: continue 

            r_new = r_new[0]
            c_new = move_assignment[r_new]
            move = (r_new, c_new)
                 
            player = "MAX" if i % 2 == 0 else "MIN"
            weight = WEIGHT_MATRIX[r_new][c_new]

            action = "TỐI ĐA hóa điểm số cho tôi" if player == "MAX" else "TỐI THIỂU hóa điểm số của đối thủ"
            
            history_log.append(f"Lượt {i+1} ({player}): Chọn ô {move} (Trọng số: {weight}) vì {action}.\n")
            
            current_state_temp = move_assignment
            
        final_assignment = best_path[-1]  
        final_board = [[0]*N for _ in range(N)]
        for r, c in final_assignment.items():
            final_board[r][c] = 1
             
        history_log.append("\n================ KẾT LUẬN =========================\n")
        history_log.append(f"Điểm Chênh Lệch Cuối Cùng (MAX - MIN): {score}\n")
        history_log.append("BÀN CỜ SAU KHI HOÀN TẤT CHUỖI NƯỚC ĐI TỐI ƯU:\n")
        history_log.append(render_board(final_board))
        
        return final_board, "".join(history_log)
    else:
        history_log.append("Không có nước đi khả dĩ.\n")
        return None, "".join(history_log)    
        
def draw_board(canvas, with_cars=False):
    global timer_running
    canvas.delete("all")
    for row in range(BOARD_SIZE):
        for col in range(BOARD_SIZE):
            x1, y1 = col * CELL_SIZE, row * CELL_SIZE
            x2, y2 = x1 + CELL_SIZE, y1 + CELL_SIZE
            color = "white" if (row + col) % 2 == 0 else "gray"
            canvas.create_rectangle(x1, y1, x2, y2,
                                    fill=color, outline="black")

    if with_cars and cars_positions:
        coords = [(r, c) for r in range(BOARD_SIZE)
                        for c in range(BOARD_SIZE)
                        if cars_positions[r][c] == 1]

        def draw_next(i=0):
            if i < len(coords):
                r, c = coords[i]
                x1, y1 = c * CELL_SIZE, r * CELL_SIZE
                canvas.create_text(x1 + CELL_SIZE/2,
                                   y1 + CELL_SIZE/2,
                                   text="♖",
                                   font=("Arial", 28),
                                   fill="red")
                canvas.after(1000, lambda: draw_next(i+1))
        draw_next()

def show_algorithm_code(event):
    global algorithm_frame, algorithm_combobox, selected
    for widget in algorithm_frame.winfo_children():
        widget.destroy()
    selected = algorithm_combobox.get()
    if selected == "Uninformed Search             ---   Breadth First Search":
        code_text = """def BFS():
    N = BOARD_SIZE
    
    # Chuyển goal_state thành tuple of tuples để so sánh
    GOAL_TUPLE = tuple(tuple(row) for row in goal_state)
    node = tuple(tuple(row) for row in original_state)
    
    path = {node: None} 
    frontier = deque([node])
    explored = {node} 

    def count_rooks(board_tuple):
        return sum(row.count(1) for row in board_tuple)

    while frontier:
        node = frontier.popleft()
        
        # 1. KIỂM TRA GOAL STATE: So sánh trực tiếp với ma trận đích
        if node == GOAL_TUPLE:
            # Tái tạo lịch sử đường đi (Logic này giữ nguyên)
            history = []
            current = node
            temp_path = []
            
            while current is not None:
                temp_path.append(current)
                current = path.get(current)
            
            temp_path.reverse()
            current_rooks = -1
            
            for step_board in temp_path:
                num_rooks = count_rooks(step_board)
                if num_rooks > current_rooks:
                    history.append(f"--- Đặt quân Xe thứ {num_rooks} ---")
                    for r_row in step_board:
                        history.append(f"{list(r_row)}")
                    current_rooks = num_rooks
            
            history.append("TÌM THẤY LỜI GIẢI HỢP LỆ!")
            return [list(row) for row in node], history

        # 2. Sinh trạng thái con (chỉ sinh khi chưa đủ N quân)
        k = count_rooks(node)
        if k == N:
            continue 

        next_row = k
        a = [list(row) for row in node]
        
        for j in range(N):
            is_col_free = True
            for r in range(next_row): 
                if a[r][j] == 1:
                    is_col_free = False
                    break
            
            if is_col_free:
                b = [list(row) for row in a]
                b[next_row][j] = 1
                child = tuple(tuple(row) for row in b)
                
                if child not in explored:
                    path[child] = node
                    explored.add(child)
                    frontier.append(child)

    return None, ["Không tìm thấy lời giải."]"""
        
    elif selected == "Uninformed Search             ---   Depth First Search":
        code_text = """def DFS():
    N = BOARD_SIZE
    
    # Chuyển goal_state thành tuple of tuples để so sánh
    GOAL_TUPLE = tuple(tuple(row) for row in goal_state)
    node = tuple(tuple(row) for row in original_state)
    
    path = {node: None}
    stack = [node]
    visited = {node} 

    def count_rooks(board_tuple):
        return sum(row.count(1) for row in board_tuple)

    while stack:
        node = stack.pop()
        
        # 1. KIỂM TRA GOAL STATE: So sánh trực tiếp với ma trận đích
        if node == GOAL_TUPLE:
            # Tái tạo lịch sử đường đi (Logic này giữ nguyên)
            history = []
            current = node
            temp_path = []
            
            while current is not None:
                temp_path.append(current)
                current = path.get(current) 
            
            temp_path.reverse()
            current_rooks = -1
            
            for step_board in temp_path:
                num_rooks = count_rooks(step_board)
                if num_rooks > current_rooks:
                    history.append(f"--- Đặt quân Xe thứ {num_rooks} ---")
                    for r_row in step_board:
                        history.append(f"{list(r_row)}")
                    current_rooks = num_rooks
            
            history.append("TÌM THẤY LỜI GIẢI HỢP LỆ!")
            return [list(row) for row in node], history
        
        # 2. Sinh trạng thái con (chỉ sinh khi chưa đủ N quân)
        k = count_rooks(node)
        if k == N:
            continue

        next_row = k
        a = [list(row) for row in node]
        
        # Duyệt ngược (reversed) là tiêu chuẩn trong DFS để tối ưu tốc độ tìm kiếm
        for j in reversed(range(N)): 
            is_col_free = True
            for r in range(next_row): 
                if a[r][j] == 1:
                    is_col_free = False
                    break
            
            if is_col_free:
                b = [list(row) for row in a]
                b[next_row][j] = 1
                child = tuple(tuple(row) for row in b)
                
                if child not in visited:
                    path[child] = node
                    visited.add(child)
                    stack.append(child)

    return None, ["Không tìm thấy lời giải."]"""
        
    elif selected == "Uninformed Search             ---   Depth Limited Search":
        code_text = """def DLS(state=None, limit=None, path=None):
    N = BOARD_SIZE
    GOAL_TUPLE = tuple(tuple(row) for row in goal_state)
    
    # Hàm hỗ trợ (Cần được định nghĩa ở phạm vi toàn cục hoặc truyền vào)
    def count_rooks(board_tuple):
        return sum(row.count(1) for row in board_tuple)
    
    # Hàm tái tạo history (Sử dụng logic của IDS/BFS)
    def _reconstruct_history(final_state, final_path, current_limit):
        history = []
        current = final_state
        temp_path = []
        
        while current is not None:
            temp_path.append(current)
            current = final_path.get(current)
        temp_path.reverse()
        
        current_rooks = -1
        for step_board in temp_path:
            num_rooks = count_rooks(step_board)
            if num_rooks > current_rooks:
                # Ghi lịch sử đúng định dạng List of Strings
                history.append(f"--- Đặt quân Xe thứ {num_rooks} ---")
                for r_row in step_board:
                    history.append(f"{list(r_row)}")
                current_rooks = num_rooks
        
        history.append(f"TÌM THẤY LỜI GIẢI BẰNG DLS (Giới hạn: {current_limit})!")
        
        # Trả về [final_board, history] để khớp với cách bạn hứng biến
        return [list(row) for row in final_state], history

    # [Phần khởi tạo]
    if state is None:
        state = tuple(tuple(row) for row in original_state)
        path = {state: None}
        if limit is None:
            limit = N

    # 1. KIỂM TRA ĐÍCH VÀ ĐIỀU KIỆN DỪNG
    if state == GOAL_TUPLE:
        # 💥 SỬA ĐỔI: Tái tạo history và trả về (final_board, history)
        return _reconstruct_history(state, path, limit) 
    elif limit == 0:
        return "cutoff", path
    
    cutoff_occurred = False
    k = count_rooks(state)
    if k == N: 
        return "failure", path
        
    next_row = k
    a = [list(row) for row in state]
    
    # 2. Sinh trạng thái con (Tối ưu hóa theo hàng)
    for j in range(N):
        is_col_free = True
        for r in range(next_row): 
            if a[r][j] == 1:
                is_col_free = False
                break
        
        if is_col_free:
            b = [list(row) for row in a]
            b[next_row][j] = 1
            child = tuple(tuple(row) for row in b)
            
            new_path = path.copy()
            new_path[child] = state
            
            result, final_path = DLS(child, limit - 1, new_path)

            if result == "cutoff":
                cutoff_occurred = True
            elif result != "failure":
                # 💥 SỬA ĐỔI: Nếu result không phải chuỗi lỗi, nó là [final_board, history]
                return result, final_path 
                
    if cutoff_occurred:
        return "cutoff", path
    else:
        return "failure", path"""
        
    elif selected == "Uninformed Search             ---   Uniform Cost Search":
        code_text = """def UCS():
    N = BOARD_SIZE
    
    # Chuyển goal_state thành tuple of tuples để so sánh
    GOAL_TUPLE = tuple(tuple(row) for row in goal_state) 
    
    def distance(p1, p2):
        return abs(p1[0] - p2[0]) + abs(p1[1] - p2[1])
        
    def count_rooks(board_tuple):
        return sum(row.count(1) for row in board_tuple)

    original_state_tuple = tuple(tuple(row) for row in original_state)
    
    que = []
    # (cost, state, positions, last_pos)
    heapq.heappush(que, (0, original_state_tuple, [], None)) 
    
    visited = {original_state_tuple: 0}
    path = {original_state_tuple: (None, 0)} 

    while que:
        cost, node, positions, last_pos = heapq.heappop(que)
        
        # 1. Kiểm tra Goal State: SO SÁNH TRỰC TIẾP VỚI GOAL_TUPLE
        if node == GOAL_TUPLE:
            # --- Tái tạo lịch sử đường đi và chi phí ---
            final_board = [list(row) for row in node]
            history = []
            current = node
            temp_path = []
            
            while current is not None:
                parent, step_cost = path.get(current, (None, 0))
                temp_path.append((current, step_cost))
                current = parent
            
            temp_path.reverse()
            
            current_rooks = -1
            total_cost = 0
            for step_board, step_cost in temp_path:
                num_rooks = count_rooks(step_board)
                
                # Chỉ thêm trạng thái khi có quân mới được đặt
                if num_rooks > current_rooks: 
                    total_cost += step_cost
                    history.append(f"--- Đặt quân Xe thứ {num_rooks} (Chi phí: {step_cost}) ---")
                    
                    for r_row in step_board:
                        history.append(f"{list(r_row)}")
                    
                    current_rooks = num_rooks
            
            history.append(f"TÌM THẤY LỜI GIẢI HỢP LỆ! TỔNG CHI PHÍ: {total_cost}")
            return final_board, total_cost, history
        
        # Bỏ qua nếu tìm thấy đường đi rẻ hơn đến trạng thái này
        if cost > visited.get(node, float('inf')):
             continue
        
        # ... (Phần còn lại của logic sinh trạng thái con) ...
        k = count_rooks(node)
        next_row = k 
        
        # DỪNG SINH TRẠNG THÁI NẾU ĐÃ ĐẶT ĐỦ N QUÂN
        # Nếu goal_state là trạng thái N quân cuối cùng, ta cần đảm bảo
        # rằng ta không cố gắng đặt thêm quân sau khi đã đặt N-1 quân.
        if k == N:
            continue

        a = [list(row) for row in node]
        
        for j in range(N):
            is_col_free = True
            for r in range(next_row): 
                if a[r][j] == 1:
                    is_col_free = False
                    break
            
            if is_col_free:
                b = [list(row) for row in a]
                b[next_row][j] = 1

                child = tuple(tuple(row) for row in b)
                new_position = (next_row, j)
                
                # Tính chi phí
                if not positions:
                    step_cost = distance((0, 0), new_position)
                else:
                    step_cost = distance(positions[-1], new_position)
                
                new_cost = cost + (step_cost if step_cost > 0 else 1)
                new_positions = positions + [new_position]

                # Thêm vào hàng đợi
                if new_cost < visited.get(child, float('inf')):
                    visited[child] = new_cost
                    path[child] = (node, step_cost) 
                    
                    heapq.heappush(que, (new_cost, child, new_positions, new_position))

    return None, 0, ["Không tìm thấy lời giải."]"""
        
    elif selected == "Uninformed Search             ---   Iterative Deepening Search":
        code_text = """def IDS(max_depth=BOARD_SIZE):
    N = BOARD_SIZE
    
    # Hàm đếm quân (phải được định nghĩa ở phạm vi toàn cục hoặc truyền vào)
    def count_rooks(board_tuple):
        return sum(row.count(1) for row in board_tuple)
    
    for depth in range(1, max_depth + 1):
        # Gọi DLS với giới hạn độ sâu tăng dần
        result, history_or_path = DLS(limit=depth) 
        
        # Kiểm tra kiểu dữ liệu: Nếu DLS thành công, nó trả về một LIST [final_board, history]
        if isinstance(result, list): 
            
            # Khai thác kết quả (chúng ta biết result là final_board)
            final_board = result
            history = history_or_path # History được trả về ở vị trí path
            
            # Cập nhật thông báo cuối cùng trong lịch sử (Optional: để khớp với độ sâu)
            history[-1] = history[-1].replace("DLS (Giới hạn:", f"IDS (Độ sâu:")
            
            # TRẢ VỀ: (Final Board, History) - Khớp với cách IDS/BFS trả về
            return final_board, history

        # Nếu thất bại hoặc cắt cụt, tiếp tục vòng lặp

    # Nếu không tìm thấy sau khi đạt max_depth
    return None, ["Không tìm thấy lời giải trong giới hạn độ sâu."]"""
        
    elif selected == "Informed Search                  ---   Greedy Search":
        code_text = """def GreedySearch():
        N = BOARD_SIZE
    
        # Khai báo các biến cần thiết từ phạm vi toàn cục
    GOAL_TUPLE = tuple(tuple(row) for row in goal_state)
    
    # Tạo dictionary {row: col} cho goal_state để tính heuristic
    goal_positions = {}
    for r in range(N):
        try:
            c = goal_state[r].index(1)
            goal_positions[r] = c
        except ValueError:
            pass 
            
    def count_rooks(board_tuple):
        \"\"\"Đếm số quân xe đã được đặt.\"\"\"
        return sum(row.count(1) for row in board_tuple)
        
    def heuristic(new_pos, next_row):
        \"\"\"Tính h(n): Khoảng cách từ vị trí Xe mới đến vị trí Xe đích của hàng đó.\"\"\"
        r, c = new_pos
        goal_col = goal_positions.get(r, N // 2)
        return abs(c - goal_col)

    original_state_tuple = tuple(tuple(row) for row in original_state)
    
    # que: (h(n), state, positions)
    # Positions: [ (r1, c1), (r2, c2), ... ] - Lưu vết đường đi
    que = []
    heapq.heappush(que, (0, original_state_tuple, [])) 
    visited = set()

    while que:
        h_val, node, positions = heapq.heappop(que)
        
        if node in visited:
            continue
        visited.add(node)
        
        # 1. KIỂM TRA GOAL STATE
        if node == GOAL_TUPLE:
            # --- TÁI TẠO LỊCH SỬ TỪ CHUỖI NƯỚC ĐI (positions) ---
            history = []
            # Bàn cờ hiện tại bắt đầu từ trạng thái rỗng
            current_board = [list(row) for row in original_state]
            
            # Xây dựng lịch sử từng bước dựa trên positions
            for i, (r, c) in enumerate(positions):
                # 💥 Tái tạo lại ma trận từ đầu để đảm bảo tính độc lập
                # (Sử dụng các nước đi tích lũy)
                current_board[r][c] = 1 
                num_rooks = i + 1
                
                # Ghi lịch sử đúng định dạng List of Strings
                history.append(f"--- Đặt quân Xe thứ {num_rooks} (h(n) cuối: {h_val}) ---")
                for r_row in current_board:
                    history.append(f"{r_row}")
            
            history.append("TÌM THẤY LỜI GIẢI HỢP LỆ!")
            
            # TRẢ VỀ: (Final Board, History)
            return [list(row) for row in node], history

        # 2. Sinh trạng thái con (Tối ưu hóa theo hàng)
        k = count_rooks(node)
        if k == N:
            continue

        next_row = k
        a = [list(row) for row in node]
        
        for j in range(N):
            # Kiểm tra cột có trống không (tính hợp lệ)
            is_col_free = True
            for r in range(next_row): 
                if a[r][j] == 1:
                    is_col_free = False
                    break
            
            if is_col_free:
                # Tạo trạng thái con
                b = [list(row) for row in a]
                b[next_row][j] = 1
                child = tuple(tuple(row) for row in b)
                
                # Tính h(n) cho trạng thái con
                new_pos = (next_row, j)
                h_n = heuristic(new_pos, next_row)
                
                new_positions = positions + [new_pos]
                
                # Chỉ thêm vào que nếu chưa được visited
                if child not in visited: 
                    heapq.heappush(que, (h_n, child, new_positions))

    return None, [\"Không tìm thấy lời giải.\"]"""

    elif selected == "Informed Search                  ---   A_star":
        code_text = """def A_star():
    def distance(p1, p2):
        # g(n) và h(n) đều dùng Manhattan distance trong ví dụ này
        return abs(p1[0] - p2[0]) + abs(p1[1] - p2[1])

    original_state_tuple = tuple(tuple(row) for row in original_state)
    que = []
    # (f(n), g(n), state, positions)
    heapq.heappush(que, (0, 0, original_state_tuple, []))
    visited = {}

    while que:
        f_val, g_val, c, positions = heapq.heappop(que)
        a = [list(row) for row in c]

        if a == goal_state:
            return a

        state_key = tuple(tuple(row) for row in a)
        if state_key in visited and visited[state_key] <= g_val:
            continue
        visited[state_key] = g_val

        for i in range(BOARD_SIZE):
            for j in range(BOARD_SIZE):
                if a[i][j] == 1:
                    continue
                b = [row.copy() for row in a]
                b[i][j] = 1
                valid = True

                for r in range(BOARD_SIZE):
                    if sum(b[r]) > 1:
                        valid = False
                        break
                for c2 in range(BOARD_SIZE):
                    if sum(b[r][c2] for r in range(BOARD_SIZE)) > 1:
                        valid = False
                        break

                if not valid:
                    continue

                state = tuple(tuple(row) for row in b)
                new_positions = positions + [(i, j)]

                # g(n): chi phí thật (dùng Manhattan giữa 2 quân liên tiếp)
                if not positions:
                    step_cost = distance((0, 0), (i, j))
                else:
                    step_cost = distance(positions[-1], (i, j))
                g_new = g_val + (step_cost if step_cost > 0 else 1)

                # h(n): ước lượng còn lại → ví dụ khoảng cách Manhattan
                # tới góc dưới phải (tùy định nghĩa goal)
                h_new = distance((i, j), (BOARD_SIZE-1, BOARD_SIZE-1))
                f_new = g_new + h_new

                heapq.heappush(que, (f_new, g_new, state, new_positions))
    return None"""

    elif selected == "Local Search                         ---   Hill_Climbing":
        code_text = """def Hill_Climbing(goal_state=goal_state, BOARD_SIZE=BOARD_SIZE):
    def conflicts(state):
        cnt = 0
        for i in range(BOARD_SIZE):
            for j in range(BOARD_SIZE):
                if state[i][j] == 1:
                    cnt += sum(state[i][k] for k in range(BOARD_SIZE)) - 1
                    cnt += sum(state[k][j] for k in range(BOARD_SIZE)) - 1
        return cnt // 2

    def random_state():
        state = [[0]*BOARD_SIZE for _ in range(BOARD_SIZE)]
        for i in range(BOARD_SIZE):
            j = random.randint(0, BOARD_SIZE-1)
            state[i][j] = 1
        return state

    current = random_state()
    current_cost = conflicts(current)
    history = [copy.deepcopy(current)]  # Lưu lại bước đầu tiên

    while True:
        neighbors = []
        for i in range(BOARD_SIZE):
            for j in range(BOARD_SIZE):
                if current[i][j] == 1:
                    for new_j in range(BOARD_SIZE):
                        if new_j != j:
                            new_state = [row[:] for row in current]
                            new_state[i][j] = 0
                            new_state[i][new_j] = 1
                            neighbors.append(new_state)

        next_state = None
        next_cost = float("inf")
        for n in neighbors:
            c = conflicts(n)
            if c < next_cost:
                next_cost = c
                next_state = n

        if next_cost >= current_cost:  # Local minima
            return current, history

        current = next_state
        current_cost = next_cost
        history.append(copy.deepcopy(current))

        if current == goal_state or current_cost == 0:
            return current, history"""

    elif selected == "Local Search                         ---   Simulated_Annealing":
        code_text = """def Simulated_Annealing(goal_state=goal_state, BOARD_SIZE=BOARD_SIZE, max_iter=100000, T=100, cooling=0.99):
    def conflicts(state):
        cnt = 0
        for i in range(BOARD_SIZE):
            for j in range(BOARD_SIZE):
                if state[i][j] == 1:
                    cnt += sum(state[i][k] for k in range(BOARD_SIZE)) - 1
                    cnt += sum(state[k][j] for k in range(BOARD_SIZE)) - 1
        return cnt // 2

    def random_state():
        state = [[0]*BOARD_SIZE for _ in range(BOARD_SIZE)]
        for i in range(BOARD_SIZE):
            j = random.randint(0, BOARD_SIZE-1)
            state[i][j] = 1
        return state

    def random_neighbor(state):
        neighbor = [row[:] for row in state]
        row = random.randint(0, BOARD_SIZE-1)
        col_old = neighbor[row].index(1)
        col_new = random.randint(0, BOARD_SIZE-1)
        while col_new == col_old:
            col_new = random.randint(0, BOARD_SIZE-1)
        neighbor[row][col_old] = 0
        neighbor[row][col_new] = 1
        return neighbor

    current = random_state()
    current_cost = conflicts(current)
    history = [current]

    for step in range(max_iter):
        if current_cost == 0 or (goal_state is not None and current == goal_state):
            return current, history

        neighbor = random_neighbor(current)
        neighbor_cost = conflicts(neighbor)
        delta = neighbor_cost - current_cost

        if delta < 0 or random.random() < math.exp(-delta / T):
            current, current_cost = neighbor, neighbor_cost
            history.append(current)

        T *= cooling
        if T < 1e-6:
            break

    return current, history"""

    elif selected == "Local Search                         ---   Genetic_Algorithm":
        code_text = """def Genetic_Algorithm(target=goal_state, N=BOARD_SIZE, pop_size=100, mutation_rate=0.1, max_gen=1000):
    history = []  # lưu các cha mẹ từng thế hệ + con đích

    def random_state():
        state = [[0] * N for _ in range(N)]
        for i in range(N):
            c = random.randint(0, N - 1)
            state[i][c] = 1
        return state

    def fitness(state):
        cost = 0
        for i in range(N):
            col_state = state[i].index(1)
            col_target = target[i].index(1)
            cost += abs(col_state - col_target)
        return cost

    def selection(population):
        total_fit = sum(1 / (1 + fitness(ind)) for ind in population)
        pick = random.uniform(0, total_fit)
        current = 0
        for ind in population:
            current += 1 / (1 + fitness(ind))
            if current > pick:
                return ind
        return population[-1]

    def crossover(p1, p2):
        point = random.randint(1, N - 2)
        child1 = [row[:] for row in p1[:point]] + [row[:] for row in p2[point:]]
        child2 = [row[:] for row in p2[:point]] + [row[:] for row in p1[point:]]
        return child1, child2

    def mutate(state):
        if random.random() < mutation_rate:
            r = random.randint(0, N - 1)
            c_old = state[r].index(1)
            c_new = random.randint(0, N - 1)
            while c_new == c_old:
                c_new = random.randint(0, N - 1)
            state[r][c_old] = 0
            state[r][c_new] = 1
        return state

    # --- main loop ---
    population = [random_state() for _ in range(pop_size)]
    best = min(population, key=fitness)

    for _ in range(max_gen):
        if best == target:
            history.append(best)  # thêm con đích vào lịch sử
            return best, history

        new_population = []
        parents_this_gen = []  # lưu cha mẹ của thế hệ này
        while len(new_population) < pop_size:
            p1 = selection(population)
            p2 = selection(population)
            parents_this_gen.extend([p1, p2])
            c1, c2 = crossover(p1, p2)
            new_population.append(mutate(c1))
            new_population.append(mutate(c2))

        history.append(parents_this_gen)  # lưu cha mẹ của thế hệ
        population = new_population
        best = min(population, key=fitness)

    history.append(best)  # nếu kết thúc mà chưa đạt target, lưu best
    return best, history"""

    elif selected == "Local Search                         ---   Beam_Search":
        code_text = """def Beam_Search(k=2):
    history_state = []  # lưu đường đi duy nhất

    def cost_function(state):
        conflicts = 0
        for i in range(len(state)):
            for j in range(i + 1, len(state)):
                if state[i] == state[j] or abs(state[i] - state[j]) == j - i:
                    conflicts += 1
        return conflicts

    beam = [(0, [])]

    for row in range(BOARD_SIZE):
        candidates = []
        for cost, state in beam:
            for col in range(BOARD_SIZE):
                if col not in state:
                    new_state = state + [col]
                    new_cost = cost_function(new_state)
                    candidates.append((new_cost, new_state))

        if not candidates:
            return None, history_state

        # Chọn k best, nhưng lưu **một state dẫn đến board tốt nhất hiện tại**
        beam = heapq.nsmallest(k, candidates, key=lambda x: x[0])
        # Chọn state tốt nhất trong beam làm bước tiếp theo của đường đi
        best_cost, best_state = min(beam, key=lambda x: x[0])
        history_state.append(best_state)

        # Nếu đã đạt goal
        if len(best_state) == BOARD_SIZE:
            board = [[0] * BOARD_SIZE for _ in range(BOARD_SIZE)]
            for r, c in enumerate(best_state):
                board[r][c] = 1
            return board, history_state

    # Nếu hết vòng lặp mà chưa đạt goal → trả best
    best_cost, best_state = min(beam, key=lambda x: x[0])
    board = [[0] * BOARD_SIZE for _ in range(BOARD_SIZE)]
    for r, c in enumerate(best_state):
        board[r][c] = 1
    return board, history_state"""
    elif selected == "Complex Environment Search    ---   DFS_And_Or":
        code_text = """    
def DFS_AND_OR():
    goal_state_tuple = tuple(tuple(row) for row in goal_state)

    # --- Hàm đệ quy nội tuyến thực hiện logic tìm kiếm cốt lõi ---
    def _search(state_tuple, path_history):
        # Nếu là trạng thái đích -> trả về đường đi chỉ chứa chính nó
        if state_tuple == goal_state_tuple:
            return [state_tuple]

        # Nếu gặp chu trình -> thất bại
        if state_tuple in path_history:
            return None

        # Logic OR: Duyệt qua mọi hành động có thể
        current_board = [list(row) for row in state_tuple]
        for r in range(BOARD_SIZE):
            for c in range(BOARD_SIZE):
                if current_board[r][c] == 0:
                    temp_board = [row[:] for row in current_board]
                    temp_board[r][c] = 1
                    is_valid_action = True
                    if sum(temp_board[r]) > 1: is_valid_action = False
                    if is_valid_action and sum(temp_board[i][c] for i in range(BOARD_SIZE)) > 1: is_valid_action = False
                    
                    if is_valid_action:
                        # Logic AND: Mọi kết quả của hành động phải giải được
                        result_state = tuple(tuple(row) for row in temp_board)
                        path_from_child = _search(result_state, path_history + [state_tuple])
                        
                        if path_from_child is not None:
                            full_path = [state_tuple] + path_from_child
                            return full_path
        
        # Nếu không có hành động nào thành công -> thất bại
        return None

    # --- Phần thân chính của hàm ---
    initial_state_tuple = tuple(tuple(row) for row in original_state)
    final_path_tuples = _search(initial_state_tuple, [])
    
    if final_path_tuples is not None:
        # 1. Bảng kết quả cuối cùng
        final_board = [list(row) for row in final_path_tuples[-1]]
        
        # 2. Lịch sử đường đi (list các bảng)
        history_state = [[list(row) for row in state] for state in final_path_tuples]
        
        return final_board, history_state
    else:
        # Trả về None và một list rỗng nếu không tìm thấy
        return None, []"""
    elif selected == "Complex Environment Search    ---   Belief_State_Search":
        code_text = """def Belief_Search(N=BOARD_SIZE):
    def valid(state):
        # Kiểm tra không có 2 xe cùng cột
        for c in range(N):
            if sum(state[r][c] for r in range(N)) > 1:
                return False
        return True

    def count_rooks(state):
        return sum(sum(row) for row in state)

    # --- Trạng thái niềm tin ban đầu ---
    s1 = [[0]*N for _ in range(N)]
    s2 = [[0]*N for _ in range(N)]
    s1[0][0], s1[1][2] = 1, 1
    s2[0][1], s2[1][3], s2[2][0] = 1, 1, 1
    belief_set = [s1, s2]

    history = "===== TẬP TRẠNG THÁI BAN ĐẦU =====\n"
    for i, s in enumerate(belief_set, 1):
        history += f"\n--- State {i} ({count_rooks(s)} xe) ---\n"
        for row in s:
            history += f"{row}\n"

    step = 1
    while True:
        # Dừng nếu tất cả state đã đủ N xe
        if all(count_rooks(s) == N for s in belief_set):
            history += "\nTất cả bàn cờ đã đủ 4 xe!\n"
            break

        history += f"\n\n===== TẬP TRẠNG THÁI BƯỚC {step} =====\n"
        new_set = []

        # --- Sinh tập trạng thái mới ---
        for idx, s in enumerate(belief_set):
            if count_rooks(s) < N:
                empty_rows = [r for r in range(N) if sum(s[r]) == 0]
                if empty_rows:
                    r = random.choice(empty_rows)
                    history += f"\nTừ State {idx+1} (chọn hàng {r+1} để sinh 4 state mới):\n"
                    for c in range(N):
                        new_state = [row[:] for row in s]
                        new_state[r][c] = 1
                        history += f"\nState mới ({r},{c}):\n"
                        for row in new_state:
                            history += f"{row}\n"
                        new_set.append(new_state)
                else:
                    new_set.append(s)
            else:
                new_set.append(s)

        # --- Kiểm tra hợp lệ ---
        valid_set = []
        history += f"\n===== ĐÁNH GIÁ SAU BƯỚC {step} =====\n"
        for i, s in enumerate(new_set, 1):
            if valid(s):
                valid_set.append(s)
                history += f"\nState {i} hợp lệ:\n"
                for row in s:
                    history += f"{row}\n"
            else:
                history += f"\nState {i} phạm luật (trùng cột):\n"
                for row in s:
                    history += f"{row}\n"

        belief_set = valid_set
        if not belief_set:
            history += "\nTất cả state bị loại → không tìm được lời giải.\n"
            return None, history

        step += 1

    # --- Kết quả cuối ---
    final = [s for s in belief_set if valid(s)]
    if final:
        history += "\nTÌM ĐƯỢC LỜI GIẢI HỢP LỆ!\n"
        for row in final[0]:
            history += f"{row}\n"
        return final[0], history
    else:
        history += "\nKhông có bàn cờ hợp lệ trong tập cuối cùng.\n"
        return None, history"""
    elif selected == "Complex Environment Search    ---   Partial_Observable_Search":
        code_text = """def Partial_Observable_Search(N=BOARD_SIZE):
    def valid(state):
        # Kiểm tra không có 2 xe cùng hàng hoặc cùng cột
        for i in range(N):
            if sum(state[i]) > 1:  # cùng hàng
                return False
            if sum(state[r][i] for r in range(N)) > 1:  # cùng cột
                return False
        return True

    def count_rooks(state):
        return sum(sum(row) for row in state)

    # --- Trạng thái quan sát được 1 phần ---
    # Chỉ biết vị trí của 1 vài xe (giống như chỉ thấy 1 phần bàn cờ)
    observed_state = [[0]*N for _ in range(N)]
    observed_state[0][1] = 1  # quan sát thấy 1 xe ở (0,1)
    observed_state[2][3] = 1  # và 1 xe ở (2,3)

    # --- Tập trạng thái khả dĩ (niềm tin) ---
    belief_set = []
    for _ in range(3):  # sinh 3 trạng thái khả dĩ khác nhau
        s = [row[:] for row in observed_state]
        # thêm ngẫu nhiên 1-2 xe ở vị trí chưa biết
        while count_rooks(s) < random.randint(2, 3):
            r, c = random.randint(0, N-1), random.randint(0, N-1)
            if s[r][c] == 0:
                s[r][c] = 1
        belief_set.append(s)

    history = "===== TRẠNG THÁI QUAN SÁT 1 PHẦN =====\n"
    for i, s in enumerate(belief_set, 1):
        history += f"\n--- State {i} ---\n"
        for row in s:
            history += f"{row}\n"

    step = 1
    while True:
        if all(count_rooks(s) == N for s in belief_set):
            history += "\nMọi bàn cờ khả dĩ đều đủ 4 xe!\n"
            break

        history += f"\n\n===== BƯỚC {step}: CẬP NHẬT NIỀM TIN =====\n"
        new_belief = []

        for idx, s in enumerate(belief_set):
            if count_rooks(s) < N:
                # chọn ngẫu nhiên 1 hàng chưa có xe để thử đặt thêm
                empty_rows = [r for r in range(N) if sum(s[r]) == 0]
                if empty_rows:
                    r = random.choice(empty_rows)
                    for c in range(N):
                        new_state = [row[:] for row in s]
                        new_state[r][c] = 1
                        new_belief.append(new_state)
                        history += f"\nTừ State {idx+1} sinh ({r},{c}):\n"
                        for row in new_state:
                            history += f"{row}\n"

        # lọc trạng thái hợp lệ
        valid_set = [s for s in new_belief if valid(s)]
        history += f"\n===== ĐÁNH GIÁ SAU BƯỚC {step} =====\n"
        for i, s in enumerate(valid_set, 1):
            history += f"\nState {i} hợp lệ:\n"
            for row in s:
                history += f"{row}\n"

        if not valid_set:
            history += "\nTất cả state bị loại → không còn khả năng quan sát hợp lệ.\n"
            return None, history

        belief_set = valid_set
        step += 1

    final = [s for s in belief_set if valid(s)]
    if final:
        history += "\nTÌM ĐƯỢC TẬP TRẠNG THÁI HỢP LỆ!\n"
        for row in final[0]:
            history += f"{row}\n"
        return final[0], history
    else:
        history += "\nKhông có trạng thái hợp lệ.\n"
        return None, history
"""
    elif selected == "Constraint Satisfaction Problem      ---   Backtracking Search":
        code_text = """def Backtrack_Search(N=BOARD_SIZE):
    variables = list(range(N))  # tập biến: các hàng
    domains = {v: list(range(N)) for v in variables}  # tập giá trị: các cột khả dĩ

    history = "===== CSP BACKTRACKING =====\n"
    history_log = [history]
    history_log.append(f"Tập biến (Rows) = {variables}\n")
    history_log.append(f"Tập giá trị (Columns) = {[i for i in range(N)]}\n\n")

    def is_consistent(assignment, var, value):
        for r, c in assignment.items():
            if c == value:  # trùng cột
                return False
        return True

    def print_state(assignment):
        board = [[0]*N for _ in range(N)]
        for r, c in assignment.items():
            board[r][c] = 1
        history_log.append("State:\n")
        for row in board:
            history_log.append(f"{row}\n")
        history_log.append("\n")

    def backtrack(assignment):
        if len(assignment) == N:
            return assignment

        # chọn hàng chưa gán ngẫu nhiên
        remaining_vars = [v for v in variables if v not in assignment]
        var = random.choice(remaining_vars)

        # chọn cột khả dĩ ngẫu nhiên
        possible_values = [v for v in domains[var] if is_consistent(assignment, var, v)]
        random.shuffle(possible_values)
        for value in possible_values:
            assignment[var] = value
            print_state(assignment)  # lưu state sau khi đặt quân
            result = backtrack(assignment)
            if result is not None:
                return result
            del assignment[var]  # quay lui
        return None

    result = backtrack({})

    if result:
        final_board = [[0]*N for _ in range(N)]
        for r, c in result.items():
            final_board[r][c] = 1
        history_log.append("TÌM ĐƯỢC LỜI GIẢI HỢP LỆ!\n")
        for row in final_board:
            history_log.append(f"{row}\n")
        return final_board, "".join(history_log)
    else:
        history_log.append("Không có lời giải.\n")
        return None, "".join(history_log)"""
    elif selected == "Constraint Satisfaction Problem      ---   Forward_Checking_Search":
        code_text = """def Forward_Checking_Search(N=BOARD_SIZE):
    variables = list(range(N))  # tập biến: các hàng
    domains = {v: list(range(N)) for v in variables}  # tập giá trị: các cột khả dĩ

    history = "===== CSP FORWARD CHECKING =====\n"
    history_log = [history]
    history_log.append(f"Tập biến (Rows) = {variables}\n")
    history_log.append(f"Tập giá trị (Columns) = {[i for i in range(N)]}\n\n")

    def is_consistent(assignment, var, value):
        for r, c in assignment.items():
            if c == value:  # trùng cột
                return False
        return True

    def print_state(assignment):
        board = [[0]*N for _ in range(N)]
        for r, c in assignment.items():
            board[r][c] = 1
        history_log.append("State:\n")
        for row in board:
            history_log.append(f"{row}\n")
        history_log.append("\n")

    def backtrack(assignment, local_domains):
        if len(assignment) == N:
            return assignment

        remaining_vars = [v for v in variables if v not in assignment]
        var = random.choice(remaining_vars)

        # chọn các giá trị hợp lệ theo miền hiện tại
        possible_values = [v for v in local_domains[var] if is_consistent(assignment, var, v)]
        random.shuffle(possible_values)

        for value in possible_values:
            assignment[var] = value
            print_state(assignment)

            # forward checking: cập nhật miền biến chưa gán
            new_domains = copy.deepcopy(local_domains)
            del new_domains[var]
            for other_var in new_domains:
                new_domains[other_var] = [v for v in new_domains[other_var] if v != value]

            # nếu có biến nào mất hết giá trị khả dĩ → quay lui ngay
            if any(len(vals) == 0 for vals in new_domains.values()):
                del assignment[var]
                continue

            result = backtrack(assignment, new_domains)
            if result is not None:
                return result

            # quay lui
            del assignment[var]

        return None

    result = backtrack({}, domains)

    if result:
        final_board = [[0]*N for _ in range(N)]
        for r, c in result.items():
            final_board[r][c] = 1
        history_log.append("TÌM ĐƯỢC LỜI GIẢI HỢP LỆ!\n")
        for row in final_board:
            history_log.append(f"{row}\n")
        return final_board, "".join(history_log)
    else:
        history_log.append("Không có lời giải.\n")
        return None, "".join(history_log)"""
    elif selected == "Constraint Satisfaction Problem      ---   AC3_Search":
        code_text = """def AC3_Search(N=BOARD_SIZE):
    variables = list(range(N))  # tập biến: các hàng
    domains = {v: list(range(N)) for v in variables}  # tập giá trị: các cột khả dĩ

    history_log = []
    history_log.append("===== CSP AC-3 =====\n")
    history_log.append(f"Tập biến (Rows) = {variables}\n")
    history_log.append(f"Tập giá trị (Columns) = {[i for i in range(N)]}\n\n")

    # Kiểm tra tính tương thích của 2 biến
    def is_consistent(var1, val1, var2, val2):
        return val1 != val2  # không trùng cột

    # AC-3: lọc miền
    def ac3(domains):
        queue = deque([(Xi, Xj) for Xi in variables for Xj in variables if Xi != Xj])
        while queue:
            Xi, Xj = queue.popleft()
            if revise(Xi, Xj, domains):
                if len(domains[Xi]) == 0:
                    return False  # miền rỗng, không có giải
                for Xk in variables:
                    if Xk != Xi and Xk != Xj:
                        queue.append((Xk, Xi))
        return True

    def revise(Xi, Xj, domains):
        revised = False
        for x in domains[Xi][:]:
            if all(not is_consistent(Xi, x, Xj, y) for y in domains[Xj]):
                domains[Xi].remove(x)
                revised = True
        return revised

    # In trạng thái miền hiện tại
    def print_state(domains):
        history_log.append("State (Domains):\n")
        for var in variables:
            history_log.append(f"Row {var}: {domains[var]}\n")
        history_log.append("\n")

    # Backtracking kết hợp AC-3
    def backtrack(assignment, local_domains):
        if len(assignment) == N:
            return assignment

        remaining_vars = [v for v in variables if v not in assignment]
        var = random.choice(remaining_vars)

        # random giá trị trong miền hiện tại
        possible_values = local_domains[var][:]
        random.shuffle(possible_values)

        for value in possible_values:
            assignment[var] = value
            new_domains = copy.deepcopy(local_domains)
            new_domains[var] = [value]

            # lọc miền bằng AC-3
            if ac3(new_domains):
                print_state(new_domains)
                result = backtrack(assignment, new_domains)
                if result is not None:
                    return result

            # quay lui
            del assignment[var]

        return None

    result = backtrack({}, domains)

    if result:
        final_board = [[0]*N for _ in range(N)]
        for r, c in result.items():
            final_board[r][c] = 1
        history_log.append("TÌM ĐƯỢC LỜI GIẢI HỢP LỆ!\n")
        for row in final_board:
            history_log.append(f"{row}\n")
        return final_board, "".join(history_log)
    else:
        history_log.append("Không có lời giải.\n")
        return None, "".join(history_log)"""
    elif selected == "Adversarial Search Algorithms      ---   Minimax_Search":
        code_text = """def Minimax_Search(N=BOARD_SIZE):
    history_log = [f"==== ADVERSARIAL MINIMAX (WEIGHTED {N}x{N} BOARD) ====\n"]
    
    # 1. KHỞI TẠO CỤC BỘ (Để đảm bảo các biến được tìm thấy)
    WEIGHT_MATRIX = [[random.randint(1, 10) for _ in range(N)] for _ in range(N)]
    history_log.append("Ma trận Trọng số (Giá trị mỗi ô):\n")
    history_log.append("\n".join([f"{row}" for row in WEIGHT_MATRIX]) + "\n")

    def render_board(matrix):
        # Hàm phụ trợ để in ra ma trận 0/1 hoặc ma trận trọng số
        return "\n".join([f"{row}" for row in matrix])

    def evaluate(state):
        max_score = 0
        min_score = 0
        sorted_items = sorted(state.items())
        
        for i, (r, c) in enumerate(sorted_items):
             # Phân biệt quân MAX/MIN theo lượt đặt
            if i % 2 == 0: 
                max_score += WEIGHT_MATRIX[r][c]
            else: 
                min_score += WEIGHT_MATRIX[r][c]

        return max_score - min_score

    def get_valid_moves(current_state):
        valid_moves = []
        assigned_rows = current_state.keys()
        assigned_cols = current_state.values()
        remaining_rows = [r for r in range(N) if r not in assigned_rows]
        if not remaining_rows: return []
        next_row = min(remaining_rows)
        for col in range(N):
            if col not in assigned_cols:
                valid_moves.append((next_row, col))
        return valid_moves

    def apply_move(current_state, move):
        r, c = move
        new_state = current_state.copy()
        new_state[r] = c
        return new_state

    # 2. HÀM MINIMAX (Tìm kiếm đường đi tối ưu)
    def minimax(current_state, depth, is_maximizing_player):
        if depth == 0 or len(current_state) == N:
            return evaluate(current_state), None, [current_state] 

        best_eval = -float('inf') if is_maximizing_player else float('inf')
        best_move = None
        best_path = []
        
        possible_moves = get_valid_moves(current_state)
        
        # Tối ưu hóa: Sắp xếp nước đi (giúp tăng tốc Alpha-Beta nếu có)
        possible_moves.sort(key=lambda move: WEIGHT_MATRIX[move[0]][move[1]], reverse=True)
        
        for move in possible_moves:
            new_state = apply_move(current_state, move)
            # Không cần ghi log đệ quy, chỉ cần tìm đường đi
            eval_score, _, path_from_child = minimax(new_state, depth - 1, not is_maximizing_player)
            
            if (is_maximizing_player and eval_score > best_eval) or \
               (not is_maximizing_player and eval_score < best_eval):
                    best_eval = eval_score
                    best_move = move
                    best_path = [new_state] + path_from_child
        
        return best_eval, best_move, best_path

    # 3. PHẦN THÂN CHÍNH (Tạo lịch sử đơn giản từ best_path)
    initial_assignment = {} 
    max_depth = N 

    score, best_move, best_path = minimax(initial_assignment, max_depth, True)
    
    # Xử lý kết quả và tạo lịch sử đơn giản
    if best_move:
        history_log.append("\n============= LỊCH SỬ CHUỖI NƯỚC ĐI TỐI ƯU =============\n")
        
        # Tạo lịch sử hành động từ chuỗi trạng thái tối ưu
        current_state_temp = initial_assignment.copy()
        
        for i, move_assignment in enumerate(best_path):
            
            # Tìm nước đi mới được thêm vào
            r_new = list(set(move_assignment.keys()) - set(current_state_temp.keys()))
            if not r_new: continue # Bỏ qua nếu không tìm thấy sự khác biệt (xảy ra ở bước đầu tiên)

            r_new = r_new[0]
            c_new = move_assignment[r_new]
            move = (r_new, c_new)
                 
            player = "MAX" if i % 2 == 0 else "MIN"
            weight = WEIGHT_MATRIX[r_new][c_new]

            action = "TỐI ĐA hóa điểm số cho tôi" if player == "MAX" else "TỐI THIỂU hóa điểm số của đối thủ"
            
            # Ghi lịch sử đơn giản
            history_log.append(f"Lượt {i+1} ({player}): Chọn ô {move} (Trọng số: {weight}) vì {action}.\n")
            
            # Cập nhật trạng thái hiện tại
            current_state_temp = move_assignment
            
        final_assignment = best_path[-1]  
        final_board = [[0]*N for _ in range(N)]
        for r, c in final_assignment.items():
            final_board[r][c] = 1
             
        history_log.append("\n============= KẾT LUẬN =====================\n")
        history_log.append(f"Điểm Chênh Lệch Cuối Cùng (MAX - MIN): {score}\n")
        history_log.append("BÀN CỜ KHI HOÀN TẤT CHUỖI NƯỚC ĐI TỐI ƯU:\n")
        history_log.append(render_board(final_board))
        
        return final_board, "".join(history_log)
    else:
        history_log.append("Không có nước đi khả dĩ.\n")
        return None, "".join(history_log)"""
    elif selected == "Adversarial Search Algorithms      ---   Alpha_Beta_Pruning_Search":
        code_text = """def Alpha_Beta_Pruning_Search(N=BOARD_SIZE):
    history_log = [f"===== ADVERSARIAL ALPHA-BETA PRUNING (WEIGHTED {N}x{N} BOARD) =====\\n"]
    
    # THÔNG BÁO VỀ KHÁC BIỆT ALPHA-BETA VÀ MINIMAX (Sự khác biệt về Hiệu suất)
    history_log.append("✅ Thuật toán này cho kết quả giống Minimax nhưng **nhanh hơn** vì đã áp dụng kỹ thuật Tỉa (Pruning).")
    history_log.append("Alpha-Beta đã **bỏ qua việc đánh giá** các nhánh (nước đi) mà không thể dẫn đến kết quả tối ưu cuối cùng.\\n")
    
    # 1. KHỞI TẠO CỤC BỘ VÀ HÀM PHỤ (Giống Minimax)
    WEIGHT_MATRIX = [[random.randint(1, 10) for _ in range(N)] for _ in range(N)]
    history_log.append("Ma trận Trọng số (Giá trị mỗi ô):\\n")
    history_log.append("\\n".join([f"{row}" for row in WEIGHT_MATRIX]) + "\\n")

    def render_board(matrix):
        return "\\n".join([f"{row}" for row in matrix])

    def evaluate(state):
        max_score = 0
        min_score = 0
        sorted_items = sorted(state.items())
        
        for i, (r, c) in enumerate(sorted_items):
            if i % 2 == 0: 
                max_score += WEIGHT_MATRIX[r][c]
            else: 
                min_score += WEIGHT_MATRIX[r][c]
        return max_score - min_score

    def get_valid_moves(current_state):
        valid_moves = []
        assigned_rows = current_state.keys()
        assigned_cols = current_state.values()
        remaining_rows = [r for r in range(N) if r not in assigned_rows]
        if not remaining_rows: return []
        next_row = min(remaining_rows)
        for col in range(N):
            if col not in assigned_cols:
                valid_moves.append((next_row, col))
        return valid_moves

    def apply_move(current_state, move):
        r, c = move
        new_state = current_state.copy()
        new_state[r] = c
        return new_state

    # 2. HÀM ALPHA-BETA PRUNING (Có logic tỉa Alpha và Beta)
    def alpha_beta(current_state, depth, alpha, beta, is_maximizing_player):
        if depth == 0 or len(current_state) == N:
            return evaluate(current_state), None, [current_state]

        best_move = None
        best_path = []
        possible_moves = get_valid_moves(current_state)
        possible_moves.sort(key=lambda move: WEIGHT_MATRIX[move[0]][move[1]], reverse=True)

        if is_maximizing_player:
            max_eval = -float('inf')
            
            for move in possible_moves:
                new_state = apply_move(current_state, move)
                eval_score, _, path_from_child = alpha_beta(new_state, depth - 1, alpha, beta, False)
                
                if eval_score > max_eval:
                    max_eval = eval_score
                    best_move = move
                    best_path = [new_state] + path_from_child

                alpha = max(alpha, max_eval)
                
                # Tỉa Beta (Beta Pruning)
                if beta <= alpha:
                    break 
            
            return max_eval, best_move, best_path
        
        else: # Minimizing Player
            min_eval = float('inf')

            for move in possible_moves:
                new_state = apply_move(current_state, move)
                eval_score, _, path_from_child = alpha_beta(new_state, depth - 1, alpha, beta, True)
                
                if eval_score < min_eval:
                    min_eval = eval_score
                    best_move = move
                    best_path = [new_state] + path_from_child

                beta = min(beta, min_eval)

                # Tỉa Alpha (Alpha Pruning)
                if beta <= alpha:
                    break 
            
            return min_eval, best_move, best_path

    # 3. PHẦN THÂN CHÍNH
    initial_assignment = {} 
    max_depth = N 

    score, best_move, best_path = alpha_beta(initial_assignment, max_depth, -float('inf'), float('inf'), True)
    
    if best_move:
        history_log.append("\\n================ LỊCH SỬ CHUỖI NƯỚC ĐI TỐI ƯU ================\\n")
        
        current_state_temp = initial_assignment.copy()
        
        for i, move_assignment in enumerate(best_path):
            r_new = list(set(move_assignment.keys()) - set(current_state_temp.keys()))
            if not r_new: continue 

            r_new = r_new[0]
            c_new = move_assignment[r_new]
            move = (r_new, c_new)
                 
            player = "MAX" if i % 2 == 0 else "MIN"
            weight = WEIGHT_MATRIX[r_new][c_new]

            action = "TỐI ĐA hóa điểm số cho tôi" if player == "MAX" else "TỐI THIỂU hóa điểm số của đối thủ"
            
            history_log.append(f"Lượt {i+1} ({player}): Chọn ô {move} (Trọng số: {weight}) vì {action}.\\n")
            
            current_state_temp = move_assignment
            
        final_assignment = best_path[-1]  
        final_board = [[0]*N for _ in range(N)]
        for r, c in final_assignment.items():
            final_board[r][c] = 1
             
        history_log.append("\\n================ KẾT LUẬN =========================\\n")
        history_log.append(f"Điểm Chênh Lệch Cuối Cùng (MAX - MIN): {score}\\n")
        history_log.append("BÀN CỜ SAU KHI HOÀN TẤT CHUỖI NƯỚC ĐI TỐI ƯU:\\n")
        history_log.append(render_board(final_board))
        
        return final_board, "".join(history_log)
    else:
        history_log.append("Không có nước đi khả dĩ.\\n")
        return None, "".join(history_log)"""

    else:
        code_text = "# Chưa có code"
        
    # Xóa widget cũ
    for widget in algorithm_frame.winfo_children():
        widget.destroy()
    
    # Tạo Text widget với scrollbar
    code_widget = tk.Text(algorithm_frame, font=("Consolas", 12), wrap="none", bg="black",fg="green")
    code_widget.insert("1.0", code_text)
    code_widget.config(state="disabled")  
    code_widget.pack(side="left", fill="both", expand=True)
    
    scrollbar = tk.Scrollbar(algorithm_frame, command=code_widget.yview)
    scrollbar.pack(side="right", fill="y")
    code_widget.config(yscrollcommand=scrollbar.set)

def show_total_state(history_state):
    global state_frame
    for widget in state_frame.winfo_children():
        widget.destroy()

    formatted_text = re.sub(r"\](?!$)", "]\n", history_state)

    code_widget = tk.Text(state_frame, font=("Consolas", 12,"bold"), wrap="char", bg="black",fg="green")
    code_widget.insert("1.0", formatted_text)  # dùng formatted_text thay vì history_state
    code_widget.config(state="disabled")  
    code_widget.pack(side="left", fill="both", expand=True)
    
    scrollbar = tk.Scrollbar(state_frame, command=code_widget.yview)
    scrollbar.pack(side="right", fill="y")
    code_widget.config(yscrollcommand=scrollbar.set)
    
def start_algo():
    global cars_positions, canvas_right_global, timer_running, timer_elapse
    timer_elapse = 0
    timer_running
    timer_running = True
    update_timer()
    if selected == "Uninformed Search             ---   Breadth First Search":
        cars_positions, history = BFS()
        history_state = "💡 Lịch sử các trạng thái bàn cờ đã đi qua:\n\n".format(len(history))
        history_state += str(history)     
    elif selected == "Uninformed Search             ---   Depth First Search":
        cars_positions, history = DFS()
        history_state = "💡 Lịch sử các trạng thái bàn cờ đã đi qua:\n\n".format(len(history))
        history_state += str(history)         
    elif selected == "Uninformed Search             ---   Uniform Cost Search":
        cars_positions,_, history = UCS()
        history_state = "💡 Lịch sử các trạng thái bàn cờ đã đi qua:\n\n".format(len(history))
        history_state += str(history)    
    elif selected == "Uninformed Search             ---   Depth Limited Search":
        cars_positions, history = DLS()
        history_state = "💡 Lịch sử các trạng thái bàn cờ đã đi qua:\n\n".format(len(history))
        history_state += str(history)
    elif selected == "Uninformed Search             ---   Iterative Deepening Search":
        cars_positions, history = IDS()
        history_state = f"💡 Lịch sử các trạng thái bàn cờ đã đi qua ({len(history)} mục):\n\n"
        history_state += "\n".join(history) # Dùng join() để lịch sử đẹp
    elif selected == "Informed Search                  ---   Greedy Search":
        cars_positions, history = GreedySearch()
        history_state = f"💡 Lịch sử các trạng thái bàn cờ đã đi qua ({len(history)} mục):\n\n"
        history_state += "\n".join(history) # Dùng join() để lịch sử đẹp
    elif selected == "Informed Search                  ---   A_star":
        cars_positions,_, history = A_star()
        history_state = f"💡 Lịch sử các trạng thái bàn cờ đã đi qua ({len(history)} mục):\n\n"
        history_state += "\n".join(history) # Dùng join() để lịch sử đẹp
    elif selected == "Local Search                         ---   Hill_Climbing":
        cars_positions, history = Hill_Climbing() 
        history_state = "💡 Lịch sử các trạng thái bàn cờ đã đi qua:\n\n".format(len(history))
        history_state += str(history)  
    elif selected == "Local Search                         ---   Simulated_Annealing":
        cars_positions, history = Simulated_Annealing() 
        history_state = "💡 Lịch sử các trạng thái bàn cờ đã đi qua:\n\n".format(len(history))
        history_state += str(history)  
    elif selected == "Local Search                         ---   Genetic_Algorithm":
        cars_positions, history = Genetic_Algorithm() 
        history_state = "Lịch sử các trạng thái bàn cờ đã đi qua:\n\n"\
                        "💡 Ma trận đầu tiên: Random\n"\
                        "💡 Ma trận cuối: Trạng thái đích\n"\
                        "💡 Các ma trận còn lại lần lượt là các cặp\n"\
                        "CHA và MẸ của từng thế hệ để tạo\n"\
                        "quần thể mới:\n\n".format(len(history))
        history_state += str(history)  
    elif selected == "Local Search                         ---   Beam_Search":
        cars_positions, history = Beam_Search() 
        history_state = "💡 Lịch sử các trạng thái bàn cờ đã đi qua:\n\n".format(len(history))
        history_state += str(history)   
    elif selected == "Complex Environment Search    ---   DFS_And_Or":
        cars_positions, history = DFS_AND_OR()
        history_state = "💡 Lịch sử các trạng thái bàn cờ đã đi qua:\n\n".format(len(history))
        history_state += str(history)
    elif selected == "Complex Environment Search    ---   Belief_State_Search":
        cars_positions, history = Belief_Search()
        history_state = "💡 Lịch sử các trạng thái bàn cờ đã đi qua:\n\n".format(len(history))
        history_state += str(history)
    elif selected == "Complex Environment Search    ---   Partial_Observable_Search":
        cars_positions, history = Partial_Observable_Search()
        history_state = "💡 Lịch sử các trạng thái bàn cờ đã đi qua:\n\n".format(len(history))
        history_state += str(history)
    elif selected == "Constraint Satisfaction Problem      ---   Backtracking Search":
        cars_positions, history = Backtrack_Search()
        history_state = "💡 Lịch sử các trạng thái bàn cờ đã đi qua:\n\n".format(len(history))
        history_state += str(history)
    elif selected == "Constraint Satisfaction Problem      ---   Forward_Checking_Search":
        cars_positions, history = Forward_Checking_Search()
        history_state = "💡 Lịch sử các trạng thái bàn cờ đã đi qua:\n\n".format(len(history))
        history_state += str(history)
    elif selected == "Constraint Satisfaction Problem      ---   AC3_Search":
        cars_positions, history = AC3_Search()
        history_state = "💡 Lịch sử các trạng thái bàn cờ đã đi qua:\n\n".format(len(history))
        history_state += str(history)
    elif selected == "Adversarial Search Algorithms      ---   Minimax_Search":
        cars_positions, history = Minimax_Search()
        history_state = "💡 Lịch sử các trạng thái bàn cờ đã đi qua:\n\n".format(len(history))
        history_state += str(history)
    elif selected == "Adversarial Search Algorithms      ---   Alpha_Beta_Pruning_Search":
        cars_positions, history = Alpha_Beta_Pruning_Search()
        history_state = "💡 Lịch sử các trạng thái bàn cờ đã đi qua:\n\n".format(len(history))
        history_state += str(history)
    else:
        cars_positions = None
  
    draw_board(canvas_right_global, with_cars=True)
    show_total_state(history_state)
    
def main():
    root=tk.Tk()
    root.title("Bài toán 8 quân xe")
    root.configure(bg="#ecf0f1")
    root.state("zoomed")
    root.resizable(False,False)

    title_label = tk.Label(root,
                           text="BÀI TOÁN 8 QUÂN XE",
                           font=("Arial",35,"bold"),
                           bg="#ecf0f1",
                           fg="#2c3e50")
    title_label.pack(side="top",pady=20) 

    # Left frame
    left_frame = tk.Frame(root, bg="#34495e")
    left_frame.place(relx=0.0, rely=0.0, anchor="nw",
                        x=80, y=180, width=800, height=830)
    
    global algorithm_combobox, algorithm_frame, state_frame
    algorithm_combobox= ttk.Combobox(left_frame, 
                                  values=["Uninformed Search             ---   Breadth First Search", 
                                          "Uninformed Search             ---   Depth First Search", 
                                          "Uninformed Search             ---   Uniform Cost Search", 
                                          "Uninformed Search             ---   Depth Limited Search",
                                          "Uninformed Search             ---   Iterative Deepening Search", 
                                          "Informed Search                  ---   Greedy Search",
                                          "Informed Search                  ---   A_star",
                                          "Local Search                         ---   Hill_Climbing",
                                          "Local Search                         ---   Simulated_Annealing",
                                          "Local Search                         ---   Genetic_Algorithm",
                                          "Local Search                         ---   Beam_Search",
                                          "Complex Environment Search    ---   DFS_And_Or",
                                         "Complex Environment Search    ---   Belief_State_Search",
                                          "Complex Environment Search    ---   Partial_Observable_Search",
                                          "Constraint Satisfaction Problem      ---   Backtracking Search",
                                          "Constraint Satisfaction Problem      ---   Forward_Checking_Search",
                                          "Constraint Satisfaction Problem      ---   AC3_Search",
                                          "Adversarial Search Algorithms      ---   Minimax_Search",
                                          "Adversarial Search Algorithms      ---   Alpha_Beta_Pruning_Search"],
                                  font=("Arial", 13, "bold"))
    algorithm_combobox.current(0)  
    algorithm_combobox.pack(side="top", fill="x", padx=10, pady=10)
    algorithm_combobox.bind("<<ComboboxSelected>>", show_algorithm_code)

    algorithm_frame = tk.Frame(left_frame, bg="black")
    algorithm_frame.pack(fill="both", expand=True, padx=15, pady=20)

    # State_frame
    state_frame = tk.Frame(root, width=400, height=250)
    state_frame.pack(side="top", anchor="ne", padx=125, pady=75, expand=True)
    state_frame.pack_propagate(False)  
    state_label = tk.Label(state_frame, text="", font=("Arial",30,"bold"),fg="red")
    state_label.pack(expand=True) 

    # Control frame
    control_frame = tk.Frame(root, bg="#34495e")
    control_frame.place(relx=1.0, rely=1.0, anchor="se",
                        y=-BOARD_SIZE*CELL_SIZE-120,
                        x=-BOARD_SIZE*CELL_SIZE+100)

    global timer_label
    timer_label = tk.Label(control_frame, text="Thời gian: 0 s",
                           font=("Arial",18))
    timer_label.grid(row=0, column=1, columnspan=4, pady=15)

    btn_start = tk.Button(control_frame, text="Bắt đầu", font=("Arial",18), command=start_algo)
    btn_start.grid(row=1, column=1, padx=15, pady=15)

    btn_reset = tk.Button(control_frame, text="Chơi lại", font=("Arial",18), command=reset_game)
    btn_reset.grid(row=1, column=3, padx=15, pady=15)
        
    # Board frame
    frame = tk.Frame(root, bg="black")
    frame.place(relx=1.0, rely=1.0, anchor="se", y=-80, x=-80)

    canvas_left = tk.Canvas(frame,
                            width= BOARD_SIZE * CELL_SIZE,
                            height= BOARD_SIZE * CELL_SIZE)
    canvas_left.grid(row=0, column=0, padx=5, pady=5)
    draw_board(canvas_left, with_cars=False)

    canvas_right = tk.Canvas(frame,
                             width = BOARD_SIZE * CELL_SIZE,
                             height = BOARD_SIZE * CELL_SIZE)
    canvas_right.grid(row=0, column=1, padx=5, pady=5)
    global canvas_right_global
    canvas_right_global = canvas_right
    draw_board(canvas_right_global, with_cars=False)

    root.mainloop()
    
if __name__ == "__main__":
    main()
 
 
 